In [1]:
import pandas as pd
import numpy as np
import os 
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
from datetime import datetime
import scipy as sp
import time
from random import random
pio.renderers.default = "iframe"

import pandas as pd
import numpy as np
import os
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
from datetime import datetime
import scipy as sp
import time
import random
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
import time 
import os
import oandapyV20
from oandapyV20 import API
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.accounts as accounts
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.trades as trades 
import oandapyV20.endpoints.positions as positions
import oandapyV20.endpoints.transactions as trans
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.definitions.primitives as primitives
import pandas as pd
from dateutil.relativedelta import relativedelta
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote_plus
import psycopg2
import os
import io
from io import StringIO 
import boto3
import pandas as pd
import numpy as np

import time 
import pygsheets
import datedelta
import calendar
import  csv
import json

from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream

from scipy import stats


def create_spike_trigger(df,col = 'delta_max',lookback_threshold = 3600):
    print('CREATING SPIKE TRIGGER...')
    df[col + '_spike_trigger'] = 0
    col_loc = df.columns.get_loc(col)
    arr = df.values
    for i in range(lookback_threshold,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            cur_max = max(arr[i - lookback_threshold : i ,col_loc])
        except:
            print('ERROR',i)
            cur_max = 100
        if arr[i,col_loc] > cur_max and sum(arr[i - 100 : i,-1]) == 0:
            arr[i,-1] = 1
    return pd.DataFrame(arr,columns = df.columns)

def create_wick_trigger(df,col = 'wick',lookback_threshold = 3600):
    
    print('CREATING WICK TRIGGER...')
    df['wick'] = 0
    df.loc[df[df['delta_vector'] < 0].index,'wick'] = df.loc[df[df['delta_vector'] < 0].index,'c'] - df.loc[df[df['delta_vector'] < 0].index,'l']
    df.loc[df[df['delta_vector'] > 0].index,'wick'] = df.loc[df[df['delta_vector'] > 0].index,'h'] - df.loc[df[df['delta_vector'] > 0].index,'c']
    df['wick_trigger'] = 0


    col_loc = df.columns.get_loc(col)
    arr = df.values
    for i in range(lookback_threshold,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            cur_max = max(arr[i - lookback_threshold : i ,col_loc])
        except:
            print('ERROR',i)
            cur_max = 100
        if arr[i,col_loc] > cur_max and sum(arr[i - 100 : i,-1]) == 0:
            arr[i,-1] = 1
    return pd.DataFrame(arr,columns = df.columns)
def consecutive_candles(df,num = 10,col = 'delta_vector'):
    print('CREATING CONSECUTIVE CANDLES TRIGGER...')
    df[col + '_consecutive_trigger'] = 0
    col_loc = df.columns.get_loc(col)
    arr = df.values
    last = -1
    counter = 0
    for i in range(arr.shape[0]):
        
        if i % 200000 == 0:
            print(i)
            
        if arr[i-1,col_loc] < 0 and arr[i,col_loc] < 0:
            counter += 1
        elif arr[i-1,col_loc] > 0 and arr[i,col_loc] > 0:
            counter += 1  
        else:
            counter = 0
            
        if counter == num:
            arr[i,-1] = 1
            counter = 0
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE OF TRIGGER DF:',df[df[col + '_consecutive_trigger'] == 1].shape)
    return df

def get_pearsons_corr(df,lookback = 10):
    print('GETTING PEARSONS CORR')
    df['ind'] = list(range(df.shape[0]))
    df['pearsons_corr'] = 0
    ind_col = df.columns.get_loc('ind')
    o_col = df.columns.get_loc('o')
    
    arr = df.values
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        r, p = sp.stats.pearsonr(arr[(i - looback) : i,ind_col], arr[(i - looback) : i,o_col])
        arr[i,-1] = r
    return pd.DataFrame(arr,columns = df.columns)

def get_pearsons_corr(df,lookback = 10):
    print('GETTING PEARSONS CORR')
    df['ind'] = list(range(df.shape[0]))
    df['pearsons_corr'] = 0
    ind_col = df.columns.get_loc('ind')
    o_col = df.columns.get_loc('o')
    
    arr = df.values
    for i in range(lookback,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        r, p = sp.stats.pearsonr(arr[(i - lookback) : i,ind_col], arr[(i - lookback) : i,o_col])
        arr[i,-1] = r
    return pd.DataFrame(arr,columns = df.columns)
def get_pearsons_corr2(df,lookback = 10):
    print('GETTING PEARSONS CORR')
    df['ind'] = list(range(df.shape[0]))
    df['pearsons_lookup'] = 0
    df['pearsons_corr2'] = 0
    ind_col = df.columns.get_loc('ind')
    o_col = df.columns.get_loc('o')
    c_col = df.columns.get_loc('c')
    new_col = df.columns.get_loc('pearsons_lookup')
    
    arr = df.values
    for i in range(lookback,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        if i % 2 == 0:
            arr[i,new_col] = arr[i,o_col]
        else:
            arr[i,new_col] = arr[i,c_col]        
            
            
    for i in range(lookback,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        r, p = sp.stats.pearsonr(arr[(i - lookback) : i,ind_col], arr[(i - lookback) : i,new_col])
        arr[i,-1] = r
    return pd.DataFrame(arr,columns = df.columns)
def convert_timestamp(df):
    time_col = df.columns.get_loc('time')
    arr = df.values 
    for i in range(arr.shape[0]):
        arr[i,time_col] = datetime.strptime(arr[i,time_col][:-4], '%Y-%m-%dT%H:%M:%S.%f')
    return pd.DataFrame(arr,columns = df.columns)
def get_best_fit(df,lookback = 10):

    def get_slope(y1,y2,total_x = 10):
        """y = mx + b"""
        return (y2 - y1) / total_x
    def get_best_fit_vals(y1,y2,total_x = 10):
        m = get_slope(y1,y2,total_x = total_x)
        lst = []
        for i in range(total_x):
            lst.append((m * i) + y1)
        return lst
    def compare_vals(lst,open_list,close_list,total_x):

        for i in range(total_x):
            if lst[i] <= max(open_list[i],close_list[i]) and lst[i] >= min(open_list[i],close_list[i]):
                pass
            else:
                return 0
        return 1
    print('GETTING BEST FIT INDICATOR')
    df['best_fit'] = 0
    o_col = df.columns.get_loc('o')
    c_col = df.columns.get_loc('c')
    new_col = df.columns.get_loc('best_fit')
    
    arr = df.values
    total_x = lookback 
    for i in range(lookback,arr.shape[0]):
        if i % 200000 == 0:
            print(i)

        y1 = np.mean([arr[i - total_x,o_col],arr[i - total_x,c_col]])
        y2 = np.mean([arr[i ,o_col],arr[i ,c_col]])
        
        temp_lst = get_best_fit_vals(y1,y2,total_x)
        arr[i,new_col] = compare_vals(lst = temp_lst,open_list = list(arr[i - total_x:i,o_col]),close_list = list(arr[i - total_x:i,c_col]),total_x = total_x)
            
    
    return pd.DataFrame(arr,columns = df.columns)   

def print_example(df,ind_list,ind = 30,delta = 240,trendline_lookback = 20):
    i = ind_list[ind]   
    print('row loc:',i,' ind:',ind,' delta:',delta)
    fig = go.Figure(data=go.Candlestick(x=df.iloc[i - delta:i + (delta*2),:]['time'],
                        open=df.iloc[i - delta:i + (delta*2),:]['o'],
                        high=df.iloc[i - delta:i + (delta*2),:]['h'],
                        low=df.iloc[i - delta:i + (delta*2),:]['l'],
                        close=df.iloc[i - delta:i + (delta*2),:]['c']))


    fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                  annotation_text="trigger point" , annotation_position="top left",
                  fillcolor="green", opacity=0.25, line_width=0)
    
    fig.add_shape(type='line',
                    x0=df.iloc[i - trendline_lookback,:]['time'],
                    y0=df.iloc[i - trendline_lookback,:]['o'],
                    x1=df.iloc[i,:]['time'],
                    y1=df.iloc[i ,:]['o'],
                    line=dict(color='Red',),
                    xref='x',
                    yref='y'
    )
    
    
    try:
        plot(fig)
    except:
        from plotly.offline import plot
        plot(fig)
def get_max_min_open_or_close(df):
    o_col = df.columns.get_loc('o')
    c_col = df.columns.get_loc('c')
    df['max_o_c'] = 0
    df['min_o_c'] = 0
    max_col = df.columns.get_loc('max_o_c')
    min_col = df.columns.get_loc('min_o_c')    
    arr = df.values
    for i in range(arr.shape[0]):
        arr[i,max_col] = max(arr[i,o_col],arr[i,c_col])
        arr[i,min_col] = min(arr[i,o_col],arr[i,c_col])
    return pd.DataFrame(arr,columns = df.columns)

def channel_indicator(df,lookback = 20):
    """Take the max of a bunch of consecutive candles and find the slope of the line
    Take the min of a bunch of consecutive candles and find the slope of the line
    
    """
    def get_slope(y1,y2,total_x = 10):
        """y = mx + b"""
        return (y2 - y1) / total_x
    def get_best_fit_vals(y1,y2,total_x = 10):
        m = get_slope(y1,y2,total_x = total_x)
        lst = []
        for i in range(total_x):
            lst.append((m * i) + y1)
        return lst
    def compare_vals(lst,check_list,total_x,check_type = 'max'):

        for i in range(total_x):
            if check_type == 'max':
                if lst[i] >= check_list[i]:
                    pass
                else:
                    return 0
            if check_type == 'min':
                if lst[i] <= check_list[i]:
                    pass
                else:
                    return 0
        return 1
    
    
    print('GETTING CHANNEL INDICATOR')
    df['channel_indicator'] = 0
    max_col = df.columns.get_loc('max_o_c')
    min_col = df.columns.get_loc('min_o_c')
    new_col = df.columns.get_loc('channel_indicator')
    df['spread_indicator'] = 0
    spread_col = df.columns.get_loc('spread_indicator')
    
    arr = df.values
    total_x = lookback 
    for i in range(lookback,arr.shape[0]):
        if i % 200000 == 0:
            print(i)

        y1 = arr[i - total_x,max_col]
        y2 = arr[i ,max_col]
        
        temp_lst1 = get_best_fit_vals(y1,y2,total_x)
        check_1 = compare_vals(lst = temp_lst1,
                                      check_list = list(arr[i - total_x:i,min_col]),
                                      total_x = total_x,
                                      check_type = 'max'
                                     )
        
        
        y1 = arr[i - total_x,min_col]
        y2 = arr[i ,min_col]
        
        temp_lst2 = get_best_fit_vals(y1,y2,total_x)
        check_2 = compare_vals(lst = temp_lst2,
                                      check_list = list(arr[i - total_x:i,max_col]),
                                      total_x = total_x,
                                      check_type = 'min'
                                     )   
        spread = np.array(temp_lst1) - np.array(temp_lst2)     
        arr[i,spread_col] = max(spread)
        
        
        if check_1 == 1 and check_2 == 1:
            arr[i,new_col] = 1
    
    return pd.DataFrame(arr,columns = df.columns)  


def get_support(df,lookup_range = 60,stop_range = 20000,lookup_range2 = 200):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    df['support_lookup'] = 0
    df['support_indicator'] = 0
    lookup_col = df.columns.get_loc('support_lookup')
    new_col = df.columns.get_loc('support_indicator')
    arr = df.values
    
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            if arr[i,c_col] == min(arr[i - lookup_range : i + lookup_range,c_col]):
                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                for j in range(i + lookup_range2,i + stop_range):
                    if arr[j,l_col] <= val and lookup_check == 0:
                        arr[j,new_col] = 1
                        arr[j,lookup_col] = lookup_ind
                        lookup_check = 1
                        break
        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df
def simulate_results(
    num_trades = 200,
    acc_val = 10000,
    rr = 6,
    risk_size = .02,
    win_rate = .2,
pr = True):
    start = acc_val
    for i in range(num_trades):
        rand = random()
        if rand > win_rate:
            acc_val = acc_val - (acc_val * risk_size)
        else:
            acc_val = acc_val + (acc_val * (risk_size * rr))
    if pr:
        print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
    return round(acc_val)


def strategy_tester_buy(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True):
    print()
    print('RUNNING BUY SIMULATOR','SL',sl,'TP',tp)
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            try:
                for j in range(i + 1,i + 10000):
                    if i not in trade_list:
                        if arr[j,l_col] < val - sl:
                            trade_res.append(-10)
                            trade_list.append(i)
                            losses.append(i)
                            break
                        if arr[j,h_col] >= val + tp:
                            trade_res.append(50)
                            trade_list.append(i)
                            wins.append(i)
                            break
            except:
                pass
    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    return np.median(vals),win_rate


def strategy_tester_sell(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True):
    print()
    print('RUNNING SELL SIMULATOR','SL',sl,'TP',tp)
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            for j in range(i + 1,i + 10000):
                if i not in trade_list:
                    if arr[j,h_col] > val + sl:
                        trade_res.append(-10)
                        trade_list.append(i)
                        losses.append(i)
                        break
                    if arr[j,l_col] <= val - tp:
                        trade_res.append(50)
                        trade_list.append(i)
                        wins.append(i)
                        break
    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    return np.median(vals),win_rate

def strategy_tester_buy(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True):
    print()
    print('RUNNING BUY SIMULATOR','SL',sl,'TP',tp)
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            trade_id = 0
            try:
                for j in range(i + 1,i + 10000):
                    if i not in trade_list:
                        if trade_id == 0:
                            if arr[j,l_col] < val - sl:
                                trade_res.append(-10)
                                trade_list.append(i)
                                losses.append(i)
                                trade_id = 1

                                break
                            if arr[j,h_col] >= val + tp:
                                trade_res.append(50)
                                trade_list.append(i)
                                wins.append(i)
                                trade_id = 1
                                break
            except:
                pass
    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    return np.median(vals),win_rate,wins,losses

def strategy_tester_sell(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True):
    print()
    print('RUNNING SELL SIMULATOR','SL',sl,'TP',tp)
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            try:
                for j in range(i + 1,i + 10000):
                    if i not in trade_list:
                        if arr[j,h_col] > val + sl:
                            trade_res.append(-10)
                            trade_list.append(i)
                            losses.append(i)
                            break
                        if arr[j,l_col] <= val - tp:
                            trade_res.append(50)
                            trade_list.append(i)
                            wins.append(i)
                            break
            except:
                pass
    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    return np.median(vals),win_rate,wins,losses
def ema(df,num = 14):
    print('GETTING EMA INDICATOR FOR:',num)
    close_col = df.columns.get_loc('c')
    df['ema_' + str(num)] = df['c']
    arr = df.values
    mult = 2/ (num + 1)
    for i in range(num,arr.shape[0]):
        sma = sum(arr[i - num + 1: i + 1,close_col]) / num
        arr[i,-1] = ((arr[i,close_col] - arr[i - 1,-1]) * mult) + arr[i - 1,-1]
    return pd.DataFrame(arr,columns = df.columns)
def strategy_tester_buy(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True):
    print()
    print('RUNNING BUY SIMULATOR','SL',sl,'TP',tp)
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            trade_id = 0
            try:
                for j in range(i + 1,i + 10000):
                    if i not in trade_list:
                        if trade_id == 0:
                            if arr[j,l_col] < val - sl:
                                trade_res.append(-10)
                                trade_list.append(i)
                                losses.append(i)
                                trade_id = 1

                                break
                            if arr[j,h_col] >= val + tp:
                                trade_res.append(50)
                                trade_list.append(i)
                                wins.append(i)
                                trade_id = 1
                                break
            except:
                pass
    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    return np.median(vals),win_rate,wins,losses

def strategy_tester_sell(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True):
    print()
    print('RUNNING SELL SIMULATOR','SL',sl,'TP',tp)
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            try:
                for j in range(i + 1,i + 10000):
                    if i not in trade_list:
                        if arr[j,h_col] > val + sl:
                            trade_res.append(-10)
                            trade_list.append(i)
                            losses.append(i)
                            break
                        if arr[j,l_col] <= val - tp:
                            trade_res.append(50)
                            trade_list.append(i)
                            wins.append(i)
                            break
            except:
                pass
    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    return np.median(vals),win_rate,wins,losses
file = 'XAU_USD_M1_2019-01-01_2022-01-31.csv'
#file = 'GBP_JPY_M1_2019-01-01_2022-01-31.csv'
#file = 'XAU_USD_S5/2020-08-01_2020-09-01.csv'
#file = 'GBP_JPY_S5/2020-08-01_2020-09-01.csv'

file = 'EUR_USD_M1_2019-01-01_2022-01-31.csv'
file = 'GBP_JPY_M1_2019-01-01_2022-01-31.csv'
file = 'EUR_USD_M1_2019-01-01_2022-01-31.csv'
file = 'GBP_JPY_M1_2019-01-01_2022-01-31.csv'

file = 'EUR_USD_M1_2019-01-01_2022-01-31.csv'


def add_delta_cols(df):
    #df['delta_vector'] = 0
    #df['delta_max_vector'] = 0

    df['delta_vector'] = 0
    df['delta_max'] = 0
    df['upper_wick'] = 0
    df['lower_wick'] = 0
    
    dv_col = df.columns.get_loc('delta_vector')
    dm_col = df.columns.get_loc('delta_max')
    uw_col = df.columns.get_loc('upper_wick')
    lw_col = df.columns.get_loc('lower_wick')
    
    o_col = df.columns.get_loc('o')
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    arr = df.values
    for i in range(arr.shape[0]):
        arr[i,dv_col] = arr[i,c_col] - arr[i,o_col]
        arr[i,dm_col] = arr[i,h_col] - arr[i,l_col]
        if arr[i,c_col] > arr[i,o_col]:
            arr[i,uw_col] = arr[i,h_col] - arr[i,c_col]
            arr[i,lw_col] = arr[i,o_col] - arr[i,l_col]
            
        elif arr[i,c_col] < arr[i,o_col]:
            arr[i,uw_col] = arr[i,h_col] - arr[i,o_col]
            arr[i,lw_col] = arr[i,c_col] - arr[i,l_col]   
            
    df = pd.DataFrame(arr,columns = df.columns)
    df['delta'] = abs(df['delta_vector'])
    
    return df
def load_df(pair = 'EUR_USD'
            ,granularity = 'M5'
            ,start = datetime(2016,1,1,0,0,0)
            ,end = datetime(2022,7,31,0,0,0)):
    dir_name = os.getcwd() + '/' + pair + '_' + granularity
    path = dir_name + '/' + str(date(start.year,start.month,start.day)) + '_' + str(date(end.year,end.month,end.day)) + '.csv'
    print('PATH:',path)
    df = pd.read_csv(path)
    df = add_delta_cols(df)
    print(df.shape)
    return df

def simulate_results(
    num_trades = 200,
    acc_val = 10000,
    rr = 6,
    risk_size = .02,
    win_rate = .2,
    pr = True):
    
    start = acc_val
    for i in range(num_trades):
        rand = random.random()
        if rand > win_rate:
            acc_val = acc_val - (acc_val * risk_size)
        else:
            acc_val = acc_val + (acc_val * (risk_size * rr))
    if pr:
        print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
    return round(acc_val)

pair = 'GBP_JPY'
pair = 'EUR_USD'
pair = 'GBP_USD'
granularity = 'M5'
#df = load_df(pair = pair,granularity = granularity,start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
#df_ = df.copy()
#granularity2 = 'M1'
#df2 = load_df(pair = pair,granularity = granularity2,start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
#df2.head()


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


# FINAL SIM

In [ ]:
def run_single_parameter_sim_loop(df,
                                 df_,
                                  pair,
                                 lookup_range,
                                    lookup_range2,
                                    stop_range,
                                    pip_threshold,
                                    support_low,
                                   bullish_ma,
                                    bullish_candle,
                                    candle_size,
                                    touch_low,
                                    midpoint_offset,
                                    bound_pips,sls,tps):
    
    def strategy_tester_buy(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True):

        col_num = df.columns.get_loc(col)
        h_col = df.columns.get_loc('h')
        l_col = df.columns.get_loc('l')
        c_col = df.columns.get_loc('c')
        entry_col = df.columns.get_loc(entry)
        arr = df.values
        temp_df = df[df[col] == 1]
 
        trade_list = []
        trade_res = []
        wins = []
        losses = []
        for i in range(arr.shape[0]):

            if arr[i,col_num] == 1:
                val = arr[i,entry_col]
                trade_id = 0
                try:
                    for j in range(i + 1,i + 20000):
                        if i not in trade_list:
                            if trade_id == 0:
                                if arr[j,l_col] <= val - sl:
                                    trade_res.append(-10)
                                    trade_list.append(i)
                                    losses.append(i)
                                    trade_id = 1

                                    break
                                elif arr[j,h_col] >= val + tp:
                                    trade_res.append(50)
                                    trade_list.append(i)
                                    wins.append(i)
                                    trade_id = 1
                                    break
                except:
                    pass
        win_rate = len(wins) / len(trade_list)
        rr = tp / sl
        vals = []


        acc_val = 10000

        for i in range(30):

            final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
            vals.append(final_val)


        #print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
        return np.median(vals),win_rate,wins,losses

    def strategy_tester_sell(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True):

        col_num = df.columns.get_loc(col)
        h_col = df.columns.get_loc('h')
        l_col = df.columns.get_loc('l')
        c_col = df.columns.get_loc('c')
        entry_col = df.columns.get_loc(entry)
        arr = df.values
        temp_df = df[df[col] == 1]
   
        trade_list = []
        trade_res = []
        wins = []
        losses = []
        for i in range(arr.shape[0]):

            if arr[i,col_num] == 1:
                val = arr[i,entry_col]
                try:
                    for j in range(i + 1,i + 20000):
                        if i not in trade_list:
                            if arr[j,h_col] >= val + sl:
                                trade_res.append(-10)
                                trade_list.append(i)
                                losses.append(i)
                                break
                            elif arr[j,l_col] <= val - tp:
                                trade_res.append(50)
                                trade_list.append(i)
                                wins.append(i)
                                break
                except:
                    pass
        win_rate = len(wins) / len(trade_list)
        rr = tp / sl
        vals = []

        acc_val = 10000

        for i in range(30):

            final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
            vals.append(final_val)


        #print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
        return np.median(vals),win_rate,wins,losses
    
    
    def get_support(df,
                    lookup_range = 1500,
                    lookup_range2 = 2400,
                    stop_range = 2200,
                    pip_threshold = .005,
                    support_low = True,
                   bullish_ma = True,
                    bullish_candle = True,
                    candle_size = .001,
                    touch_low = True,
                    midpoint_offset = 0,
                    bound_pips = .001
                   ):
        print('GETTING SUPPORT INDICATOR')
        s = time.time()
        c_col = df.columns.get_loc('c')
        o_col = df.columns.get_loc('o')
        l_col = df.columns.get_loc('l')
        h_col = df.columns.get_loc('h')
        uw_col = df.columns.get_loc('upper_wick')
        lw_col = df.columns.get_loc('lower_wick')
        dv_col = df.columns.get_loc('delta_vector')
        dm_col = df.columns.get_loc('delta_max')
        d_col = df.columns.get_loc('delta')
        ma1_col = df.columns.get_loc('smma_21')
        ma2_col = df.columns.get_loc('smma_50')
        ma3_col = df.columns.get_loc('smma_200')    

        df['support_lookup'] = 0
        df['support_indicator'] = 0

        lookup_col = df.columns.get_loc('support_lookup')
        new_col = df.columns.get_loc('support_indicator')

        if support_low:
            reversal_col = df.columns.get_loc('l')
        else:
            reversal_col = df.columns.get_loc('c')

        if touch_low:
            touch_col = df.columns.get_loc('l')
        else:
            touch_col = df.columns.get_loc('c')    

        arr = df.values
        min_count = 0

        print('STOP RANGE:',stop_range)
        c = 0 

    #['time', 'volume', 'o', 'h', 'l', 'c', 'utc_timestamp', 'est_timestamp',
     #      'smma_21', 'smma_50', 'smma_200', 'ema_42', 'ema_150', 'ema_600',
      #     'bearish_engulfing', 'bullish_engulfing', 'bearish_tls', 'bullish_tls',
       #    'new_york', 'london', 'delta_vector', 'delta_max', 'upper_wick',
        #   'lower_wick', 'delta']

        for i in range(arr.shape[0]):
           # if i % 200000 == 0:
            #    print(i)
            try:
                #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
                min_ = min(arr[i - lookup_range : i + lookup_range2,reversal_col])

                # If the current val is at that reversal point:
                # Check and make sure we have an adequate movement off of that reversal point
                # this is defined by the pip_range and pip threshold, pip threshold is a user defined parameter
                if arr[i,reversal_col] == min_:
                    lookup_ind = i
                    val = arr[i,reversal_col]
                    lookup_check = 0
                    max_ = max(arr[i : i + lookup_range2,c_col])
                    pip_range = max_ - min_
                    c = c + 1
                    if pip_threshold <= pip_range:
                        # Now that all of these conditions are met
                        # We need to look into the future to see if we hit this reversal level, this is defined by the stop range
                        for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                            #If all of the following conditions are met, we will mark the array with a 1 signal to indicated we would place a trade                       
                            #CANDLE BULL OR BEAR
                            if arr[j,c_col] >= arr[j,o_col]:
                                bull = True
                            else:
                                bull = False
                            #MOVING AVG INDICATOR
                            if bullish_ma == True:
                                if arr[j,ma1_col] > arr[j,ma2_col] and arr[j,ma2_col] > arr[j,ma3_col]:
                                    ma_indicator = 1
                                else:
                                    ma_indicator = 0
                            elif bullish_ma == False:
                                if arr[j,ma1_col] < arr[j,ma2_col] and arr[j,ma2_col] < arr[j,ma3_col]:
                                    ma_indicator = 1
                                else:
                                    ma_indicator = 0                        
                            else:
                                ma_indicator = 1

                            midpoint = val + midpoint_offset
                            lower_bound = midpoint - bound_pips
                            upper_bound = midpoint + bound_pips

                            if lookup_check == 0 and \
                            ma_indicator == 1 and\
                            (bull == bullish_candle or bullish_candle == None) and\
                            arr[j,d_col] >= candle_size and\
                            (arr[j,touch_col] >= lower_bound and arr[j,touch_col] <= upper_bound):

                                arr[j,new_col] = 1
                                arr[j,lookup_col] = lookup_ind
                                lookup_check = 1
                                break

            except:
                pass

        print('COUNT:',c)

        e = time.time()
        print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
        df = pd.DataFrame(arr,columns = df.columns)
        print('SHAPE',df[df['support_indicator'] == 1].shape)
        return df


    def get_resistance(df,
                    lookup_range = 1500,
                    lookup_range2 = 2400,
                    stop_range = 2200,
                    pip_threshold = .005,
                    support_low = True,
                   bullish_ma = True,
                    bullish_candle = True,
                    candle_size = .001,
                    touch_low = True,
                    midpoint_offset = 0,
                    bound_pips = .001
                   ):
        print('GETTING RESISTANCE INDICATOR')
        s = time.time()
        c_col = df.columns.get_loc('c')
        o_col = df.columns.get_loc('o')
        l_col = df.columns.get_loc('l')
        h_col = df.columns.get_loc('h')
        uw_col = df.columns.get_loc('upper_wick')
        lw_col = df.columns.get_loc('lower_wick')
        dv_col = df.columns.get_loc('delta_vector')
        dm_col = df.columns.get_loc('delta_max')
        d_col = df.columns.get_loc('delta')
        ma1_col = df.columns.get_loc('smma_21')
        ma2_col = df.columns.get_loc('smma_50')
        ma3_col = df.columns.get_loc('smma_200')    

        df['resistance_lookup'] = 0
        df['resistance_indicator'] = 0

        lookup_col = df.columns.get_loc('resistance_lookup')
        new_col = df.columns.get_loc('resistance_indicator')

        if support_low:
            reversal_col = df.columns.get_loc('h')
        else:
            reversal_col = df.columns.get_loc('c')

        if touch_low:
            touch_col = df.columns.get_loc('h')
        else:
            touch_col = df.columns.get_loc('c')    

        arr = df.values
        min_count = 0
        c = 0
        for i in range(arr.shape[0]):

            try:
                #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
                max_ = max(arr[i - lookup_range : i + lookup_range2,reversal_col])

                # If the current val is at that reversal point:
                # Check and make sure we have an adequate movement off of that reversal point
                # this is defined by the pip_range and pip threshold, pip threshold is a user defined parameter
                if arr[i,reversal_col] == max_:
                    lookup_ind = i
                    val = arr[i,reversal_col]
                    lookup_check = 0
                    min_ = min(arr[i : i + lookup_range2,c_col])
                    pip_range = max_ - min_
                    c = c + 1
                    if pip_threshold <= pip_range:
                        # Now that all of these conditions are met
                        # We need to look into the future to see if we hit this reversal level, this is defined by the stop range
                        for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                            #If all of the following conditions are met, we will mark the array with a 1 signal to indicated we would place a trade                       
                            #CANDLE BULL OR BEAR
                            if arr[j,c_col] >= arr[j,o_col]:
                                bull = True
                            else:
                                bull = False
                            #MOVING AVG INDICATOR
                            if bullish_ma == True:
                                if arr[j,ma1_col] > arr[j,ma2_col] and arr[j,ma2_col] > arr[j,ma3_col]:
                                    ma_indicator = 1
                                else:
                                    ma_indicator = 0
                            elif bullish_ma == False:
                                if arr[j,ma1_col] < arr[j,ma2_col] and arr[j,ma2_col] < arr[j,ma3_col]:
                                    ma_indicator = 1
                                else:
                                    ma_indicator = 0                        
                            else:
                                ma_indicator = 1

                            midpoint = val + midpoint_offset
                            lower_bound = midpoint - bound_pips
                            upper_bound = midpoint + bound_pips

                            if lookup_check == 0 and \
                            ma_indicator == 1 and\
                            (bull == bullish_candle or bullish_candle == None) and\
                            arr[j,d_col] >= candle_size and\
                            (arr[j,touch_col] >= lower_bound and arr[j,touch_col] <= upper_bound):

                                arr[j,new_col] = 1
                                arr[j,lookup_col] = lookup_ind
                                lookup_check = 1
                                break

            except:
                pass

        e = time.time()
        df = pd.DataFrame(arr,columns = df.columns)
        print('SHAPE',df[df['resistance_indicator'] == 1].shape)
        return df

    s = time.time()
    print('RUNNING SINGLE PARAMETER LOOP FOR PARAMS:',
          'lookup_range',lookup_range,
          'lookup_range2', lookup_range2,
                    'stop_range', stop_range,
                    'pip_threshold' , pip_threshold,
                    'support_low' , support_low,
                   'bullish_ma' , bullish_ma,
                    'bullish_candle' , bullish_candle,
                    'candle_size' , candle_size,
                    'touch_low' , touch_low,
                    'midpoint_offset' , midpoint_offset,
                    'bound_pips' , bound_pips)
    
    cols = [
        'instrument'
        ,'timeframe'
        ,'buy_or_sell'
        ,'trade_strategy'
       ,'median_sim_ending_val'
        ,'median_sim_ending_val2'
        ,'total_sim_val'
        ,'starting_val'
        ,'sl'
        ,'tp'
        ,'RR'
        ,'win_rate'    
        ,'num_trades'
        ,'risk_size' 
        ,'actual_number_of_trades'   
        ,'buy_or_sell2'
        ,'trade_strategy2'

        ,'win_rate2'    
        ,'actual_number_of_trades2'    
        ,'lookup_range' 
        ,'lookup_range2' 
        ,'stop_range' 
        ,'pip_threshold' 
        ,'support_low' 
        ,'bullish_ma' 
        ,'bullish_candle'
        ,'candle_size' 
        ,'touch_low' 
        ,'midpoint_offset' 
        ,'bound_pips' 

    ]

    df = get_support(df,
                    lookup_range = lookup_range,
                    lookup_range2 = lookup_range2,
                    stop_range = stop_range,
                    pip_threshold = pip_threshold,
                    support_low = support_low,
                   bullish_ma = bullish_ma,
                    bullish_candle = bullish_candle,
                    candle_size = candle_size,
                    touch_low = touch_low,
                    midpoint_offset = midpoint_offset,
                    bound_pips = bound_pips
                   )

    if bullish_ma == True:
        res_bullish_ma = False
    elif bullish_ma == False:
        res_bullish_ma = True
    else:
        res_bullish_ma = None

    if bullish_candle == True:
        res_bullish_candle = False
    elif bullish_candle == False:
        res_bullish_candle = True
    else:
        res_bullish_candle = None

    res_midpoint_offset = midpoint_offset * -1

    df_ = get_resistance(df_,
                    lookup_range = lookup_range,
                    lookup_range2 = lookup_range2,
                    stop_range = stop_range,
                    pip_threshold = pip_threshold,
                    support_low = support_low,
                   bullish_ma = res_bullish_ma,
                    bullish_candle = res_bullish_candle,
                    candle_size = candle_size,
                    touch_low = touch_low,
                    midpoint_offset = res_midpoint_offset,
                    bound_pips = bound_pips
                   )

    trade_strategy = 'support_indicator'
    trade_strategy2 = 'resistance_indicator'

    #trade_strategy = 'resistance_indicatorc'
    num_trades = 400
    risk_size = .01

    starting_val = 10000
    instrument = pair
    timeframe = 'M5'
    temp_lst = []
    actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
    actual_number_of_trades2 = df_[df_[trade_strategy2] == 1].shape[0]
    for sl in sls:
        for tp in tps:
            ending_val,win_rate,wins,losses = strategy_tester_buy(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)
            ending_val2,win_rate2,wins2,losses2 = strategy_tester_sell(df_,col = trade_strategy2,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)

            lst = [instrument,
                    timeframe,
                    'buy',
                    trade_strategy,
                    ending_val,
                    ending_val2,
                   ending_val + ending_val2 - starting_val,
                    starting_val,
                    sl,
                    tp,
                    round(tp / sl,4),
                    round(win_rate,4),
                    num_trades,
                    risk_size,
                    actual_number_of_trades,
                   'sell',
                    trade_strategy2,

                    win_rate2,
                    actual_number_of_trades2,
                    lookup_range,
                    lookup_range2,
                    stop_range,
                    pip_threshold,
                    support_low ,
                    bullish_ma ,
                    bullish_candle ,
                    candle_size ,
                    touch_low ,
                    midpoint_offset,
                    bound_pips 
            ]

            temp_lst.append(lst)


            ending_val,win_rate,wins,losses = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)
            ending_val2,win_rate2,wins2,losses2 = strategy_tester_buy(df_,col = trade_strategy2,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)

            lst = [instrument,
                    timeframe,
                    'sell',
                    trade_strategy,
                    ending_val,
                    ending_val2,
                   ending_val + ending_val2 - starting_val,
                    starting_val,
                    sl,
                    tp,
                    round(tp / sl,4),
                    round(win_rate,4),
                    num_trades,
                    risk_size,
                    actual_number_of_trades,
                   'buy',
                    trade_strategy2,
                    win_rate2,
                    actual_number_of_trades2,
                    lookup_range,
                    lookup_range2,
                    stop_range,
                    pip_threshold,
                    support_low ,
                    bullish_ma ,
                    bullish_candle ,
                    candle_size ,
                    touch_low ,
                    midpoint_offset,
                    bound_pips 
            ]

            temp_lst.append(lst)




    final_df = pd.DataFrame(temp_lst,columns = cols)
    final_df = final_df.sort_values(by = 'total_sim_val',ascending = False)
    
    support_best = final_df.sort_values(by = 'median_sim_ending_val',ascending = False)
    resistance_best = final_df.sort_values(by = 'median_sim_ending_val2',ascending = False)
    overall_best = final_df.sort_values(by = 'total_sim_val',ascending = False)

    print('SUPPORT BEST BUY/VAL/WR/RR/TP/SL/NUM_TRADES:',support_best['buy_or_sell'].iloc[0],'/',support_best['median_sim_ending_val'].iloc[0],'/',support_best['win_rate'].iloc[0],'/',support_best['RR'].iloc[0],'/',support_best['tp'].iloc[0],'/',support_best['sl'].iloc[0],'/',support_best['actual_number_of_trades'].iloc[0]
         )
    print('RESISTANCE BEST BUY/VAL/WR/RR/TP/SL/NUM_TRADES:',resistance_best['buy_or_sell'].iloc[0],'/',resistance_best['median_sim_ending_val2'].iloc[0],'/',resistance_best['win_rate2'].iloc[0],'/',resistance_best['RR'].iloc[0],'/',resistance_best['tp'].iloc[0],'/',resistance_best['sl'].iloc[0],'/',resistance_best['actual_number_of_trades2'].iloc[0]
         )
    print('OVERALL BEST BUY/VAL/WR/RR/TP/SL/NUM_TRADES:',overall_best['buy_or_sell'].iloc[0],'/',overall_best['total_sim_val'].iloc[0],'/',overall_best['win_rate'].iloc[0],'/',overall_best['RR'].iloc[0],'/',overall_best['tp'].iloc[0],'/',overall_best['sl'].iloc[0],'/',overall_best['actual_number_of_trades'].iloc[0]
         )


    e = time.time()
    print('TOTAL PARAM SIM LOOP TIME:',round((e-s)/60,2),'MINUTES')
    return final_df


def generate_random_param_list():
    def return_random_list_element(lst):
        return lst[random.randint(0,len(lst) - 1)]
    # param loop
    # PARAM DEFINITON
    if 'JPY' in pair:
        pip_thresholds = [.3,.5,.7]
        sls = [.05,.075,.1,.125,.15,.2,.25]
        tps = [.1,.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3]
        candle_sizes = [.05,.075,.1,.15]
        midpoint_offsets = [0,-.05,.05,-.1,.1,-.15,.15,-.25,.25]
        bound_pips_ = [.025,.05,.075,.1,.15]
    else:
        pip_thresholds = [.003,.005,.007]
        sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
        tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013]  
        candle_sizes = [.0005,.00075,.001,.0015]
        midpoint_offsets = [0,-.0005,.0005,-.001,.001,-.0015,.0015,-.0025,.0025]
        bound_pips_ = [.00025,.0005,.00075,.001,.0015]

    lookup_ranges = [100,288,400,600]
    lookup_ranges2 = [100,288,400,600]    
    stop_ranges = [1000,3000,6000,8000]

    support_lows = [True,False]
    bullish_mas = [True,False,None]
    bullish_candles = [True,False,None]
    touch_lows = [True,False]
    
    lookup_range = return_random_list_element(lookup_ranges)
    lookup_range2 = return_random_list_element(lookup_ranges2)
    stop_range = return_random_list_element(stop_ranges)
    pip_threshold = return_random_list_element(pip_thresholds)
    support_low = return_random_list_element(support_lows)
    bullish_ma = return_random_list_element(bullish_mas)
    bullish_candle = return_random_list_element(bullish_candles)
    candle_size = return_random_list_element(candle_sizes)
    touch_low = return_random_list_element(touch_lows)
    midpoint_offset = return_random_list_element(midpoint_offsets)
    bound_pips = return_random_list_element(bound_pips_)
    param_check_list = [lookup_range,lookup_range2,stop_range,pip_threshold,support_low,bullish_ma,bullish_candle,candle_size,touch_low,midpoint_offset,bound_pips]
    
    return param_check_list,lookup_range,lookup_range2,stop_range,pip_threshold,support_low,bullish_ma,bullish_candle,candle_size,touch_low,midpoint_offset,bound_pips,sls,tps



def run_sim(pair,max_loops):
    print('RUNNING SIM...')
    file = pair + '_M1_2019-01-01_2022-01-31.csv'
    df = load_df(pair = pair,granularity ='M5',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
    df_ = df.copy()
    file_name = 'SIM_CLEAN_' + file
    file_name2 = 'CHECK_PARAMS_' + file
    print(file_name,file_name2)
    try:
        final_df = pd.read_csv(file_name)
        loop_number = final_df['loop_number'].max()

        param_check_df = pd.read_csv(file_name2)
        param_check_lists = param_check_df.values.tolist()

    except:
        final_df = pd.DataFrame([0])
        param_check_df = pd.DataFrame([0])
        loop_number = 0
        param_check_lists = []
    
    while max_loops >= loop_number:
        checker = 0
        while checker == 0:
            param_check_list,lookup_range,lookup_range2,stop_range,pip_threshold,support_low,bullish_ma,bullish_candle,candle_size,touch_low,midpoint_offset,bound_pips,sls,tps = generate_random_param_list()
            if param_check_list not in param_check_lists:
                param_check_lists.append(param_check_list)

                checker = 1
        try:
            if final_df.shape[0] == 1:
                loop_number = 1
            else:
                loop_number = final_df['loop_number'].max() + 1

            print('LOOP NUMBER:',loop_number)
            loop_df = run_single_parameter_sim_loop(df,
                                                 df_,
                                                  pair,
                                                 lookup_range,
                                                    lookup_range2,
                                                    stop_range,
                                                    pip_threshold,
                                                    support_low,
                                                   bullish_ma,
                                                    bullish_candle,
                                                    candle_size,
                                                    touch_low,
                                                    midpoint_offset,
                                                    bound_pips,sls,tps)
            loop_df['loop_number'] = loop_number

            if final_df.shape[0] == 1:
                final_df = loop_df

            else:
                final_df = final_df.append(loop_df)

            final_df.to_csv(file_name,index = False)
            param_check_df = pd.DataFrame(param_check_lists)
            param_check_df.to_csv(file_name2,index = False)
       
        except:
            print('ERROR WITH THESE PARAMETERS... SKIPPING')

        support_best = final_df.sort_values(by = 'median_sim_ending_val',ascending = False)
        resistance_best = final_df.sort_values(by = 'median_sim_ending_val2',ascending = False)
        overall_best = final_df.sort_values(by = 'total_sim_val',ascending = False)

        print('OVR SUPPORT BEST BUY/VAL/WR/RR/TP/SL/NUM_TRADES:',support_best['buy_or_sell'].iloc[0],'/',support_best['median_sim_ending_val'].iloc[0],'/',support_best['win_rate'].iloc[0],'/',support_best['RR'].iloc[0],'/',support_best['tp'].iloc[0],'/',support_best['sl'].iloc[0],'/',support_best['actual_number_of_trades'].iloc[0]
             )
        print('OVR RESISTANCE BEST BUY/VAL/WR/RR/TP/SL/NUM_TRADES:',resistance_best['buy_or_sell'].iloc[0],'/',resistance_best['median_sim_ending_val2'].iloc[0],'/',resistance_best['win_rate2'].iloc[0],'/',resistance_best['RR'].iloc[0],'/',resistance_best['tp'].iloc[0],'/',resistance_best['sl'].iloc[0],'/',resistance_best['actual_number_of_trades2'].iloc[0]
             )
        print('OVR OVERALL BEST BUY/VAL/WR/RR/TP/SL/NUM_TRADES:',overall_best['buy_or_sell'].iloc[0],'/',overall_best['total_sim_val'].iloc[0],'/',overall_best['win_rate'].iloc[0],'/',overall_best['RR'].iloc[0],'/',overall_best['tp'].iloc[0],'/',overall_best['sl'].iloc[0],'/',overall_best['actual_number_of_trades'].iloc[0]
             )
        print()
        print()
        print()
        
pair = 'GBP_USD'
run_sim(pair = pair,max_loops = 2500)




RUNNING SIM...
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/GBP_USD_M5/2016-01-01_2022-07-31.csv
(489547, 25)
SIM_CLEAN_GBP_USD_M1_2019-01-01_2022-01-31.csv CHECK_PARAMS_GBP_USD_M1_2019-01-01_2022-01-31.csv
LOOP NUMBER: 830
RUNNING SINGLE PARAMETER LOOP FOR PARAMS: lookup_range 400 lookup_range2 400 stop_range 1000 pip_threshold 0.005 support_low False bullish_ma None bullish_candle None candle_size 0.00075 touch_low False midpoint_offset 0.0005 bound_pips 0.0015
GETTING SUPPORT INDICATOR
STOP RANGE: 1000
COUNT: 424
TOTAL FUNCTION TIME: 0.3003516674041748  MINUTES
SHAPE (208, 27)
GETTING RESISTANCE INDICATOR
SHAPE (203, 27)
SUPPORT BEST BUY/VAL/WR/RR/TP/SL/NUM_TRADES: sell / 74117.5 / 0.1875 / 7.0 / 0.007 / 0.001 / 208
RESISTANCE BEST BUY/VAL/WR/RR/TP/SL/NUM_TRADES: buy / 84597.0 / 0.15763546798029557 / 9.3333 / 0.007 / 0.00075 / 203
OVERALL BEST BUY/VAL/WR/RR/TP/SL/NUM_TRADES: buy / 83220.0 / 0.0962 / 9.3333 / 0.007 / 0.00075 / 208
TOTAL PARAM SIM LOOP TIME: 1.71

In [ ]:


def get_support(df,lookup_range = 1500,lookup_range2 = 2400,stop_range = 2200,pip_threshold = .005,pip_threshold2 = .005,trading_session = 'london',signal =  'bullish_engulfing'):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    df['support_lookup'] = 0
    df['support_indicator'] = 0

    lookup_col = df.columns.get_loc('support_lookup')
    new_col = df.columns.get_loc('support_indicator')
    trading_session_col = df.columns.get_loc(trading_session)
    signal_col = df.columns.get_loc(signal)
    arr = df.values
    min_count = 0
    print('STOP RANGE:',stop_range)
    c = 0 
    #for i in range(100000):
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range2,l_col])
            
            if arr[i,l_col] == min_:
               # print(min_, max(arr[i : i + stop_range,c_col]),'RANGE:',i + 30,i + stop_range)

                lookup_ind = i
                val = arr[i,l_col]
                lookup_check = 0
                max_ = max(arr[i : i + lookup_range2,c_col])
                pip_range = max_ - min_
                c = c + 1
                if pip_threshold <=  pip_range:
                    
                    for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                        #print('VAL',(max(arr[i : j,c_col]) - min_))
                        if arr[j,l_col] <= val - pip_threshold2 and lookup_check == 0:# and arr[j,trading_session_col] == 1:
                       
   
                            arr[j,new_col] = 1
                            arr[j,lookup_col] = lookup_ind
                            lookup_check = 1
                            break
                      
        except:
            pass
        
    print('COUNT:',c)
    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df








if 'JPY' in pair:
    pip_threshold = .3
    pip_threshold = .1
    pip_threshold2 = 0
    sls = [.05,.075,.1,.125,.15,.2,.25]
    tps = [.1,.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3]
else:
    pip_threshold = .005
    pip_threshold2 = .001
    sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
    tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013]  
#pip_threshold2s = [.0005,.001,.0015,.002,.0025]
#pip_thresholds = [.003,.005,.0075]
#lookup_ranges = [500,1000,1500,2000,2500]
#lookup_range2s = [500,1000,1500,2000,2500]
#stop_ranges = [12500,10000,7500,5000,2500]


cols = [
    'instrument'
    ,'timeframe'
    ,'buy_or_sell'
    ,'trade_strategy'
   ,'median_sim_ending_val'
    ,'starting_val'
    ,'sl'
    ,'tp'
    ,'RR'
    ,'win_rate'
    ,'pip_threshold'
    ,'pip_threshold2'
    ,'lookup_range'
    ,'lookup_range2'
    ,'stop_range'
    ,'num_trades'
    ,'risk_size' 
    ,'actual_number_of_trades'
    
]


lookup_range = 288 # 1 day
#lookup_range = 450
lookup_range2 = 288 # 1 day
lookup_range2 = 288
stop_range = 3000

signal = 'bullish_engulfing'

signal = 'bullish_tls'
signal = 'bearish_tls'
signal = 'bearish_engulfing'
signal = 'bearish_tls'
signal = 'bullish_tls'
signal = 'bearish_engulfing'
signal = 'bullish_engulfing'



buy_or_sell = 1
df = get_support(df,lookup_range = lookup_range,lookup_range2 = lookup_range2,stop_range = stop_range,
                  pip_threshold = pip_threshold,pip_threshold2 = pip_threshold2,trading_session = 'london',signal =  signal)
    

trade_strategy = 'support_indicator'
#trade_strategy = 'resistance_indicatorc'
num_trades = 400
risk_size = .01
actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
starting_val = 10000
instrument = pair
timeframe = 'M5'
temp_lst = []
for sl in sls:
    for tp in tps:
        ending_val,win_rate,wins,losses = strategy_tester_buy(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)

        lst = [instrument,timeframe,'buy',trade_strategy,ending_val,starting_val,sl,tp,round(tp / sl,4),round(win_rate,4),
         pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,num_trades,risk_size,actual_number_of_trades]
        temp_lst.append(lst)

        ending_val,win_rate,wins,losses = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)

        lst = [instrument,timeframe,'sell',trade_strategy,ending_val,starting_val,sl,tp,round(tp / sl,4),round(win_rate,4),
         pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,num_trades,risk_size,actual_number_of_trades]
        temp_lst.append(lst)

final_df = pd.DataFrame(temp_lst,columns = cols)
final_df = final_df.sort_values(by = 'median_sim_ending_val',ascending = False)
final_df.head(10)


In [ ]:
def get_support(df,lookup_range = 1500,lookup_range2 = 2400,stop_range = 2200,pip_threshold = .005,pip_threshold2 = .005,trading_session = 'london',signal =  'bullish_engulfing'):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    df['support_lookup'] = 0
    df['support_indicator'] = 0

    lookup_col = df.columns.get_loc('support_lookup')
    new_col = df.columns.get_loc('support_indicator')
    trading_session_col = df.columns.get_loc(trading_session)
    signal_col = df.columns.get_loc(signal)
    arr = df.values
    min_count = 0
    print('STOP RANGE:',stop_range)
    c = 0 
    #for i in range(100000):
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range2,l_col])
            
            if arr[i,l_col] == min_:
               # print(min_, max(arr[i : i + stop_range,c_col]),'RANGE:',i + 30,i + stop_range)

                lookup_ind = i
                val = arr[i,l_col]
                lookup_check = 0
                max_ = max(arr[i : i + lookup_range2,c_col])
                pip_range = max_ - min_
                c = c + 1
                if pip_threshold <=  pip_range:
                    
                    for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                        #print('VAL',(max(arr[i : j,c_col]) - min_))
                        if arr[j,l_col] <= val + pip_threshold2 and lookup_check == 0:# and arr[j,trading_session_col] == 1:
                            for k in range(j + 1, j + 48):
                                if arr[k,o_col] - arr[k,c_col] >= .0015 and arr[k,o_col] > val - .002 and arr[k,c_col] < val:
                       
   
                                    arr[k,new_col] = 1
                                    arr[k,lookup_col] = lookup_ind
                                    lookup_check = 1
                                    break
                      
        except:
            pass
        
    print('COUNT:',c)
    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df








if 'JPY' in pair:
    pip_threshold = .3
    pip_threshold = .1
    pip_threshold2 = 0
    sls = [.05,.075,.1,.125,.15,.2,.25]
    tps = [.1,.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3]
else:
    pip_threshold = .005
    pip_threshold2 = .0005
    sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
    tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013]  
#pip_threshold2s = [.0005,.001,.0015,.002,.0025]
#pip_thresholds = [.003,.005,.0075]
#lookup_ranges = [500,1000,1500,2000,2500]
#lookup_range2s = [500,1000,1500,2000,2500]
#stop_ranges = [12500,10000,7500,5000,2500]


cols = [
    'instrument'
    ,'timeframe'
    ,'buy_or_sell'
    ,'trade_strategy'
   ,'median_sim_ending_val'
    ,'starting_val'
    ,'sl'
    ,'tp'
    ,'RR'
    ,'win_rate'
    ,'pip_threshold'
    ,'pip_threshold2'
    ,'lookup_range'
    ,'lookup_range2'
    ,'stop_range'
    ,'num_trades'
    ,'risk_size' 
    ,'actual_number_of_trades'
    
]


lookup_range = 288 # 1 day
lookup_range = 100
lookup_range2 = 288 # 1 day
lookup_range2 = 100
stop_range = 2000

signal = 'bullish_engulfing'

signal = 'bullish_tls'
signal = 'bearish_tls'
signal = 'bearish_engulfing'
signal = 'bearish_tls'
signal = 'bullish_tls'
signal = 'bearish_engulfing'
signal = 'bullish_engulfing'



buy_or_sell = 1
df = get_support(df,lookup_range = lookup_range,lookup_range2 = lookup_range2,stop_range = stop_range,
                  pip_threshold = pip_threshold,pip_threshold2 = pip_threshold2,trading_session = 'london',signal =  signal)
    

trade_strategy = 'support_indicator'
#trade_strategy = 'resistance_indicatorc'
num_trades = 400
risk_size = .01
actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
starting_val = 10000
instrument = pair
timeframe = 'M5'
temp_lst = []
for sl in sls:
    for tp in tps:
        ending_val,win_rate,wins,losses = strategy_tester_buy(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)

        lst = [instrument,timeframe,'buy',trade_strategy,ending_val,starting_val,sl,tp,round(tp / sl,4),round(win_rate,4),
         pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,num_trades,risk_size,actual_number_of_trades]
        temp_lst.append(lst)

        ending_val,win_rate,wins,losses = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)

        lst = [instrument,timeframe,'sell',trade_strategy,ending_val,starting_val,sl,tp,round(tp / sl,4),round(win_rate,4),
         pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,num_trades,risk_size,actual_number_of_trades]
        temp_lst.append(lst)

final_df = pd.DataFrame(temp_lst,columns = cols)
final_df = final_df.sort_values(by = 'median_sim_ending_val',ascending = False)
final_df.head(10)


In [ ]:
def get_support(df,lookup_range = 1500,lookup_range2 = 2400,stop_range = 2200,pip_threshold = .005,pip_threshold2 = .005,trading_session = 'london',signal =  'bullish_engulfing'):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    df['support_lookup'] = 0
    df['support_indicator'] = 0

    lookup_col = df.columns.get_loc('support_lookup')
    new_col = df.columns.get_loc('support_indicator')
    trading_session_col = df.columns.get_loc(trading_session)
    signal_col = df.columns.get_loc(signal)
    arr = df.values
    min_count = 0
    print('STOP RANGE:',stop_range)
    c = 0 
    #for i in range(100000):
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range2,l_col])
            
            if arr[i,l_col] == min_:
               # print(min_, max(arr[i : i + stop_range,c_col]),'RANGE:',i + 30,i + stop_range)

                lookup_ind = i
                val = arr[i,l_col]
                lookup_check = 0
                max_ = max(arr[i : i + lookup_range2,c_col])
                pip_range = max_ - min_
                c = c + 1
                if pip_threshold <=  pip_range:
                    
                    for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                        #print('VAL',(max(arr[i : j,c_col]) - min_))
                        if arr[j,l_col] <= val + pip_threshold2 and min(arr[i:j,c_col]) >= val - .0025 and arr[j,c_col] - arr[j,o_col] >= .001 and lookup_check == 0:# and arr[j,trading_session_col] == 1:

                            arr[j,new_col] = 1
                            arr[j,lookup_col] = lookup_ind
                            lookup_check = 1
                            break
                      
        except:
            pass
        
    print('COUNT:',c)
    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df








if 'JPY' in pair:
    pip_threshold = .3
    pip_threshold = .1
    pip_threshold2 = 0
    sls = [.05,.075,.1,.125,.15,.2,.25]
    tps = [.1,.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3]
else:
    pip_threshold = .005
    pip_threshold2 = .002
    sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
    tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013]  
#pip_threshold2s = [.0005,.001,.0015,.002,.0025]
#pip_thresholds = [.003,.005,.0075]
#lookup_ranges = [500,1000,1500,2000,2500]
#lookup_range2s = [500,1000,1500,2000,2500]
#stop_ranges = [12500,10000,7500,5000,2500]


cols = [
    'instrument'
    ,'timeframe'
    ,'buy_or_sell'
    ,'trade_strategy'
   ,'median_sim_ending_val'
    ,'starting_val'
    ,'sl'
    ,'tp'
    ,'RR'
    ,'win_rate'
    ,'pip_threshold'
    ,'pip_threshold2'
    ,'lookup_range'
    ,'lookup_range2'
    ,'stop_range'
    ,'num_trades'
    ,'risk_size' 
    ,'actual_number_of_trades'
    
]


lookup_range = 288 # 1 day
lookup_range = 100
lookup_range2 = 288 # 1 day
lookup_range2 = 100
stop_range = 5000

signal = 'bullish_engulfing'

signal = 'bullish_tls'
signal = 'bearish_tls'
signal = 'bearish_engulfing'
signal = 'bearish_tls'
signal = 'bullish_tls'
signal = 'bearish_engulfing'
signal = 'bullish_engulfing'



buy_or_sell = 1
df = get_support(df,lookup_range = lookup_range,lookup_range2 = lookup_range2,stop_range = stop_range,
                  pip_threshold = pip_threshold,pip_threshold2 = pip_threshold2,trading_session = 'london',signal =  signal)
    

trade_strategy = 'support_indicator'
#trade_strategy = 'resistance_indicatorc'
num_trades = 400
risk_size = .01
actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
starting_val = 10000
instrument = pair
timeframe = 'M5'
temp_lst = []
for sl in sls:
    for tp in tps:
        ending_val,win_rate,wins,losses = strategy_tester_buy(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)

        lst = [instrument,timeframe,'buy',trade_strategy,ending_val,starting_val,sl,tp,round(tp / sl,4),round(win_rate,4),
         pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,num_trades,risk_size,actual_number_of_trades]
        temp_lst.append(lst)

        ending_val,win_rate,wins,losses = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)

        lst = [instrument,timeframe,'sell',trade_strategy,ending_val,starting_val,sl,tp,round(tp / sl,4),round(win_rate,4),
         pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,num_trades,risk_size,actual_number_of_trades]
        temp_lst.append(lst)

final_df = pd.DataFrame(temp_lst,columns = cols)
final_df = final_df.sort_values(by = 'median_sim_ending_val',ascending = False)
final_df.head(10)




In [ ]:
def get_support(df,lookup_range = 1500,lookup_range2 = 2400,stop_range = 2200,pip_threshold = .005,pip_threshold2 = .005,trading_session = 'london',signal =  'bullish_engulfing'):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    df['support_lookup'] = 0
    df['support_indicator'] = 0

    lookup_col = df.columns.get_loc('support_lookup')
    new_col = df.columns.get_loc('support_indicator')
    trading_session_col = df.columns.get_loc(trading_session)
    signal_col = df.columns.get_loc(signal)
    arr = df.values
    min_count = 0
    print('STOP RANGE:',stop_range)
    c = 0 
    #for i in range(100000):
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range2,l_col])
            
            if arr[i,l_col] == min_:
               # print(min_, max(arr[i : i + stop_range,c_col]),'RANGE:',i + 30,i + stop_range)

                lookup_ind = i
                val = arr[i,l_col]
                lookup_check = 0
                max_ = max(arr[i : i + lookup_range2,c_col])
                pip_range = max_ - min_
                c = c + 1
                if pip_threshold <=  pip_range:
                    
                    for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                        #print('VAL',(max(arr[i : j,c_col]) - min_))
                        if arr[j,l_col] <= val + pip_threshold2 and lookup_check == 0:
                            for k in range(j+1,j + 60):
                                if arr[k,c_col] <= val + .00025 and arr[k,c_col] >= val - .0015 and arr[k,o_col] - arr[k,c_col] >= .001:
  
                                    arr[k,new_col] = 1
                                    arr[k,lookup_col] = lookup_ind
                                    lookup_check = 1
                                    break
                      
        except:
            pass
        
    print('COUNT:',c)
    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df








if 'JPY' in pair:
    pip_threshold = .3
    pip_threshold = .1
    pip_threshold2 = 0
    sls = [.05,.075,.1,.125,.15,.2,.25]
    tps = [.1,.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3]
else:
    pip_threshold = .005
    pip_threshold2 = .0003
    sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
    tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013]  
#pip_threshold2s = [.0005,.001,.0015,.002,.0025]
#pip_thresholds = [.003,.005,.0075]
#lookup_ranges = [500,1000,1500,2000,2500]
#lookup_range2s = [500,1000,1500,2000,2500]
#stop_ranges = [12500,10000,7500,5000,2500]


cols = [
    'instrument'
    ,'timeframe'
    ,'buy_or_sell'
    ,'trade_strategy'
   ,'median_sim_ending_val'
    ,'starting_val'
    ,'sl'
    ,'tp'
    ,'RR'
    ,'win_rate'
    ,'pip_threshold'
    ,'pip_threshold2'
    ,'lookup_range'
    ,'lookup_range2'
    ,'stop_range'
    ,'num_trades'
    ,'risk_size' 
    ,'actual_number_of_trades'
    
]


lookup_range = 288 # 1 day
#lookup_range = 100
lookup_range2 = 288 # 1 day
lookup_range2 = 100
stop_range = 5000

signal = 'bullish_engulfing'

signal = 'bullish_tls'
signal = 'bearish_tls'
signal = 'bearish_engulfing'
signal = 'bearish_tls'
signal = 'bullish_tls'
signal = 'bearish_engulfing'
signal = 'bullish_engulfing'



buy_or_sell = 1
df = get_support(df,lookup_range = lookup_range,lookup_range2 = lookup_range2,stop_range = stop_range,
                  pip_threshold = pip_threshold,pip_threshold2 = pip_threshold2,trading_session = 'london',signal =  signal)
    

trade_strategy = 'support_indicator'
#trade_strategy = 'resistance_indicatorc'
num_trades = 400
risk_size = .01
actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
starting_val = 10000
instrument = pair
timeframe = 'M5'
temp_lst = []
for sl in sls:
    for tp in tps:
        ending_val,win_rate,wins,losses = strategy_tester_buy(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)

        lst = [instrument,timeframe,'buy',trade_strategy,ending_val,starting_val,sl,tp,round(tp / sl,4),round(win_rate,4),
         pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,num_trades,risk_size,actual_number_of_trades]
        temp_lst.append(lst)

        ending_val,win_rate,wins,losses = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)

        lst = [instrument,timeframe,'sell',trade_strategy,ending_val,starting_val,sl,tp,round(tp / sl,4),round(win_rate,4),
         pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,num_trades,risk_size,actual_number_of_trades]
        temp_lst.append(lst)

final_df = pd.DataFrame(temp_lst,columns = cols)
final_df = final_df.sort_values(by = 'median_sim_ending_val',ascending = False)
final_df.head(10)


In [ ]:
def get_support(df,lookup_range = 1500,lookup_range2 = 2400,stop_range = 2200,pip_threshold = .005,pip_threshold2 = .005,trading_session = 'london',signal =  'bullish_engulfing'):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    df['support_lookup'] = 0
    df['support_indicator'] = 0

    lookup_col = df.columns.get_loc('support_lookup')
    new_col = df.columns.get_loc('support_indicator')
    trading_session_col = df.columns.get_loc(trading_session)
    signal_col = df.columns.get_loc(signal)
    arr = df.values
    min_count = 0
    print('STOP RANGE:',stop_range)
    c = 0 
    #for i in range(100000):
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range2,l_col])
            
            if arr[i,l_col] == min_:
               # print(min_, max(arr[i : i + stop_range,c_col]),'RANGE:',i + 30,i + stop_range)

                lookup_ind = i
                val = arr[i,l_col]
                lookup_check = 0
                max_ = max(arr[i : i + lookup_range2,c_col])
                pip_range = max_ - min_
                c = c + 1
                if pip_threshold <=  pip_range:
                    
                    for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                        #print('VAL',(max(arr[i : j,c_col]) - min_))
                        if arr[j,o_col] <= val + .001 and arr[j,c_col] >= val -  .001  and arr[j,c_col] - arr[j,o_col] >= .001 and lookup_check == 0 and arr[j+1,c_col] - arr[j+1,o_col] >= .0001 and arr[j+2,o_col] - arr[j+2,c_col] >= .0001:
                        #if arr[j,l_col] <= val and arr[j,c_col] >= val - pip_threshold2  and arr[j+10,c_col] - arr[j,o_col] >= .0025 and arr[j+10,c_col] >= val and lookup_check == 0:
                            arr[j+2,new_col] = 1
                            arr[j+2,lookup_col] = lookup_ind
                            lookup_check = 1
                            break
                      
        except:
            pass
        
    print('COUNT:',c)
    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df








if 'JPY' in pair:
    pip_threshold = .3
    pip_threshold = .1
    pip_threshold2 = 0
    sls = [.05,.075,.1,.125,.15,.2,.25]
    tps = [.1,.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3]
else:
    pip_threshold = .005
    pip_threshold2 = .0025
    sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
    tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013]  
#pip_threshold2s = [.0005,.001,.0015,.002,.0025]
#pip_thresholds = [.003,.005,.0075]
#lookup_ranges = [500,1000,1500,2000,2500]
#lookup_range2s = [500,1000,1500,2000,2500]
#stop_ranges = [12500,10000,7500,5000,2500]


cols = [
    'instrument'
    ,'timeframe'
    ,'buy_or_sell'
    ,'trade_strategy'
   ,'median_sim_ending_val'
    ,'starting_val'
    ,'sl'
    ,'tp'
    ,'RR'
    ,'win_rate'
    ,'pip_threshold'
    ,'pip_threshold2'
    ,'lookup_range'
    ,'lookup_range2'
    ,'stop_range'
    ,'num_trades'
    ,'risk_size' 
    ,'actual_number_of_trades'
    
]


lookup_range = 288 # 1 day
lookup_range = 100
lookup_range2 = 288 # 1 day
lookup_range2 = 100
stop_range = 4000

signal = 'bullish_engulfing'

signal = 'bullish_tls'
signal = 'bearish_tls'
signal = 'bearish_engulfing'
signal = 'bearish_tls'
signal = 'bullish_tls'
signal = 'bearish_engulfing'
signal = 'bullish_engulfing'



buy_or_sell = 1
df = get_support(df,lookup_range = lookup_range,lookup_range2 = lookup_range2,stop_range = stop_range,
                  pip_threshold = pip_threshold,pip_threshold2 = pip_threshold2,trading_session = 'london',signal =  signal)
    

trade_strategy = 'support_indicator'
#trade_strategy = 'resistance_indicatorc'
num_trades = 400
risk_size = .01
actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
starting_val = 10000
instrument = pair
timeframe = 'M5'
temp_lst = []
for sl in sls:
    for tp in tps:
        ending_val,win_rate,wins,losses = strategy_tester_buy(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)

        lst = [instrument,timeframe,'buy',trade_strategy,ending_val,starting_val,sl,tp,round(tp / sl,4),round(win_rate,4),
         pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,num_trades,risk_size,actual_number_of_trades]
        temp_lst.append(lst)

        ending_val,win_rate,wins,losses = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)

        lst = [instrument,timeframe,'sell',trade_strategy,ending_val,starting_val,sl,tp,round(tp / sl,4),round(win_rate,4),
         pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,num_trades,risk_size,actual_number_of_trades]
        temp_lst.append(lst)

final_df = pd.DataFrame(temp_lst,columns = cols)
final_df = final_df.sort_values(by = 'median_sim_ending_val',ascending = False)
final_df.head(10)


In [ ]:
240 * .15

In [ ]:
c = 0
final_list = []
pip_threshold2s = [.0005,.001,.0015,.002,.0025]
pip_thresholds = [.003,.005,.0075]
lookup_ranges = [500,1000,1500,2000,2500]
lookup_range2s = [500,1000,1500,2000,2500]
stop_ranges = [12500,10000,7500,5000,2500]


pip_threshold2s = [.0005,.001,.0015,.002,.0025]
pip_thresholds = [.003,.005,.0075]
lookup_ranges = [1500,2000,2500]
lookup_range2s = [1000,1500,2000]
stop_ranges = [10000]

sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
tps = [.0015,.002,.0025,.003,.004,.005,.007,.009]
num_trades = 200
risk_size = .01
for pip_threshold in pip_thresholds:
    for pip_threshold2 in pip_threshold2s:
        for lookup_range in lookup_ranges:
            for lookup_range2 in lookup_range2s:
                for stop_range in stop_ranges:

# PARAMETERS
## lookup range, lookup range 2,stop range, how large the candle is, bullish vs bearish, support defined by low or close, 
## upper and lower range of tap window, midpoint of tap window relative to support, sls and tps, next candle bullish vs bearish? (or other confluence signal?)

In [ ]:
def strategy_tester_buy(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True):
    print()
    print('RUNNING BUY SIMULATOR','SL',sl,'TP',tp)
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            trade_id = 0
            try:
                for j in range(i + 1,i + 20000):
                    if i not in trade_list:
                        if trade_id == 0:
                            if arr[j,l_col] <= val - sl:
                                trade_res.append(-10)
                                trade_list.append(i)
                                losses.append(i)
                                trade_id = 1

                                break
                            elif arr[j,h_col] >= val + tp:
                                trade_res.append(50)
                                trade_list.append(i)
                                wins.append(i)
                                trade_id = 1
                                break
            except:
                pass
    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
    vals = []


    acc_val = 10000

    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    return np.median(vals),win_rate,wins,losses

def strategy_tester_sell(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True):
    print()
    print('RUNNING SELL SIMULATOR','SL',sl,'TP',tp)
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            try:
                for j in range(i + 1,i + 20000):
                    if i not in trade_list:
                        if arr[j,h_col] >= val + sl:
                            trade_res.append(-10)
                            trade_list.append(i)
                            losses.append(i)
                            break
                        elif arr[j,l_col] <= val - tp:
                            trade_res.append(50)
                            trade_list.append(i)
                            wins.append(i)
                            break
            except:
                pass
    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
    vals = []

    acc_val = 10000

    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)


    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    return np.median(vals),win_rate,wins,losses

def get_support(df,
                lookup_range = 1500,
                lookup_range2 = 2400,
                stop_range = 2200,
                pip_threshold = .005,
                support_low = True,
               bullish_ma = True,
                bullish_candle = True,
                candle_size = .001,
                touch_low = True,
                midpoint_offset = 0,
                bound_pips = .001
               ):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    uw_col = df.columns.get_loc('upper_wick')
    lw_col = df.columns.get_loc('lower_wick')
    dv_col = df.columns.get_loc('delta_vector')
    dm_col = df.columns.get_loc('delta_max')
    d_col = df.columns.get_loc('delta')
    ma1_col = df.columns.get_loc('smma_21')
    ma2_col = df.columns.get_loc('smma_50')
    ma3_col = df.columns.get_loc('smma_200')    
    
    df['support_lookup'] = 0
    df['support_indicator'] = 0

    lookup_col = df.columns.get_loc('support_lookup')
    new_col = df.columns.get_loc('support_indicator')
    
    if support_low:
        reversal_col = df.columns.get_loc('l')
    else:
        reversal_col = df.columns.get_loc('c')
        
    if touch_low:
        touch_col = df.columns.get_loc('l')
    else:
        touch_col = df.columns.get_loc('c')    

    arr = df.values
    min_count = 0
    
    print('STOP RANGE:',stop_range)
    c = 0 
    
#['time', 'volume', 'o', 'h', 'l', 'c', 'utc_timestamp', 'est_timestamp',
 #      'smma_21', 'smma_50', 'smma_200', 'ema_42', 'ema_150', 'ema_600',
  #     'bearish_engulfing', 'bullish_engulfing', 'bearish_tls', 'bullish_tls',
   #    'new_york', 'london', 'delta_vector', 'delta_max', 'upper_wick',
    #   'lower_wick', 'delta']

    for i in range(arr.shape[0]):
       # if i % 200000 == 0:
        #    print(i)
        try:
            #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
            min_ = min(arr[i - lookup_range : i + lookup_range2,reversal_col])
            
            # If the current val is at that reversal point:
            # Check and make sure we have an adequate movement off of that reversal point
            # this is defined by the pip_range and pip threshold, pip threshold is a user defined parameter
            if arr[i,reversal_col] == min_:
                lookup_ind = i
                val = arr[i,reversal_col]
                lookup_check = 0
                max_ = max(arr[i : i + lookup_range2,c_col])
                pip_range = max_ - min_
                c = c + 1
                if pip_threshold <= pip_range:
                    # Now that all of these conditions are met
                    # We need to look into the future to see if we hit this reversal level, this is defined by the stop range
                    for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                        #If all of the following conditions are met, we will mark the array with a 1 signal to indicated we would place a trade                       
                        #CANDLE BULL OR BEAR
                        if arr[j,c_col] >= arr[j,o_col]:
                            bull = True
                        else:
                            bull = False
                        #MOVING AVG INDICATOR
                        if bullish_ma == True:
                            if arr[j,ma1_col] > arr[j,ma2_col] and arr[j,ma2_col] > arr[j,ma3_col]:
                                ma_indicator = 1
                            else:
                                ma_indicator = 0
                        elif bullish_ma == False:
                            if arr[j,ma1_col] < arr[j,ma2_col] and arr[j,ma2_col] < arr[j,ma3_col]:
                                ma_indicator = 1
                            else:
                                ma_indicator = 0                        
                        else:
                            ma_indicator = 1
                        
                        midpoint = val + midpoint_offset
                        lower_bound = midpoint - bound_pips
                        upper_bound = midpoint + bound_pips

                        if lookup_check == 0 and \
                        ma_indicator == 1 and\
                        (bull == bullish_candle or bullish_candle == None) and\
                        arr[j,d_col] >= candle_size and\
                        (arr[j,touch_col] >= lower_bound and arr[j,touch_col] <= upper_bound):

                            arr[j,new_col] = 1
                            arr[j,lookup_col] = lookup_ind
                            lookup_check = 1
                            break
                      
        except:
            pass
        
    print('COUNT:',c)

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df


def get_resistance(df,
                lookup_range = 1500,
                lookup_range2 = 2400,
                stop_range = 2200,
                pip_threshold = .005,
                support_low = True,
               bullish_ma = True,
                bullish_candle = True,
                candle_size = .001,
                touch_low = True,
                midpoint_offset = 0,
                bound_pips = .001
               ):
    print('GETTING RESISTANCE INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    uw_col = df.columns.get_loc('upper_wick')
    lw_col = df.columns.get_loc('lower_wick')
    dv_col = df.columns.get_loc('delta_vector')
    dm_col = df.columns.get_loc('delta_max')
    d_col = df.columns.get_loc('delta')
    ma1_col = df.columns.get_loc('smma_21')
    ma2_col = df.columns.get_loc('smma_50')
    ma3_col = df.columns.get_loc('smma_200')    
    
    df['resistance_lookup'] = 0
    df['resistance_indicator'] = 0

    lookup_col = df.columns.get_loc('resistance_lookup')
    new_col = df.columns.get_loc('resistance_indicator')
    
    if support_low:
        reversal_col = df.columns.get_loc('h')
    else:
        reversal_col = df.columns.get_loc('c')
        
    if touch_low:
        touch_col = df.columns.get_loc('h')
    else:
        touch_col = df.columns.get_loc('c')    

    arr = df.values
    min_count = 0
    c = 0
    for i in range(arr.shape[0]):

        try:
            #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
            max_ = max(arr[i - lookup_range : i + lookup_range2,reversal_col])
            
            # If the current val is at that reversal point:
            # Check and make sure we have an adequate movement off of that reversal point
            # this is defined by the pip_range and pip threshold, pip threshold is a user defined parameter
            if arr[i,reversal_col] == max_:
                lookup_ind = i
                val = arr[i,reversal_col]
                lookup_check = 0
                min_ = min(arr[i : i + lookup_range2,c_col])
                pip_range = max_ - min_
                c = c + 1
                if pip_threshold <= pip_range:
                    # Now that all of these conditions are met
                    # We need to look into the future to see if we hit this reversal level, this is defined by the stop range
                    for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                        #If all of the following conditions are met, we will mark the array with a 1 signal to indicated we would place a trade                       
                        #CANDLE BULL OR BEAR
                        if arr[j,c_col] >= arr[j,o_col]:
                            bull = True
                        else:
                            bull = False
                        #MOVING AVG INDICATOR
                        if bullish_ma == True:
                            if arr[j,ma1_col] > arr[j,ma2_col] and arr[j,ma2_col] > arr[j,ma3_col]:
                                ma_indicator = 1
                            else:
                                ma_indicator = 0
                        elif bullish_ma == False:
                            if arr[j,ma1_col] < arr[j,ma2_col] and arr[j,ma2_col] < arr[j,ma3_col]:
                                ma_indicator = 1
                            else:
                                ma_indicator = 0                        
                        else:
                            ma_indicator = 1
                        
                        midpoint = val + midpoint_offset
                        lower_bound = midpoint - bound_pips
                        upper_bound = midpoint + bound_pips

                        if lookup_check == 0 and \
                        ma_indicator == 1 and\
                        (bull == bullish_candle or bullish_candle == None) and\
                        arr[j,d_col] >= candle_size and\
                        (arr[j,touch_col] >= lower_bound and arr[j,touch_col] <= upper_bound):

                            arr[j,new_col] = 1
                            arr[j,lookup_col] = lookup_ind
                            lookup_check = 1
                            break
                      
        except:
            pass
        
    e = time.time()
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['resistance_indicator'] == 1].shape)
    return df



cols = [
    'instrument'
    ,'timeframe'
    ,'buy_or_sell'
    ,'trade_strategy'
   ,'median_sim_ending_val'
    ,'median_sim_ending_val2'
    ,'total_sim_val'
    ,'starting_val'
    ,'sl'
    ,'tp'
    ,'RR'
    ,'win_rate'    
    ,'num_trades'
    ,'risk_size' 
    ,'actual_number_of_trades'   
    ,'buy_or_sell2'
    ,'trade_strategy2'
   
    ,'win_rate2'    
    ,'actual_number_of_trades2'    
    ,'lookup_range' 
    ,'lookup_range2' 
    ,'stop_range' 
    ,'pip_threshold' 
    ,'support_low' 
    ,'bullish_ma' 
    ,'bullish_candle'
    ,'candle_size' 
    ,'touch_low' 
    ,'midpoint_offset' 
    ,'bound_pips' 
    
]




# PARAM DEFINITON
if 'JPY' in pair:
    pip_threshold = .5
    sls = [.05,.075,.1,.125,.15,.2,.25]
    tps = [.1,.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3]
    candle_sizes = [.05,.075,.1,.15]
    midpoint_offsets = [0,-.05,.05,-.1,.1,-.15,.15,-.25,.25]
    bound_pips_ = [.025,.05,.075,.1,.15]
else:
    pip_threshold = .005
    sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
    tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013]  
    candle_sizes = [.0005,.00075,.001,.0015]
    midpoint_offsets = [0,-.0005,.0005,-.001,.001,-.0015,.0015,-.0025,.0025]
    bound_pips_ = [.00025,.0005,.00075,.001,.0015]
    
lookup_ranges = [100,288,400,600]
lookup_ranges2 = [100,288,400,600]    
stop_ranges = [1000,3000,6000,8000]

support_lows = [True,False]
bullish_mas = [True,False,None]
bullish_candles = [True,False,None]
touch_lows = [True,False]

lookup_range = 250
lookup_range2 = 150
stop_range = 4500
pip_threshold = pip_threshold
support_low = True
bullish_ma = None
bullish_candle = False
candle_size = candle_sizes[2]
touch_low = touch_lows[0]
midpoint_offset = midpoint_offsets[0]
bound_pips = bound_pips_[3]


df = get_support(df,
                lookup_range = lookup_range,
                lookup_range2 = lookup_range2,
                stop_range = stop_range,
                pip_threshold = pip_threshold,
                support_low = support_low,
               bullish_ma = bullish_ma,
                bullish_candle = bullish_candle,
                candle_size = candle_size,
                touch_low = touch_low,
                midpoint_offset = midpoint_offset,
                bound_pips = bound_pips
               )

if bullish_ma == True:
    res_bullish_ma = False
elif bullish_ma == False:
    res_bullish_ma = True
else:
    res_bullish_ma = None
    
if bullish_candle == True:
    res_bullish_candle = False
elif bullish_candle == False:
    res_bullish_candle = True
else:
    res_bullish_candle = None

res_midpoint_offset = midpoint_offset * -1

df_ = get_resistance(df_,
                lookup_range = lookup_range,
                lookup_range2 = lookup_range2,
                stop_range = stop_range,
                pip_threshold = pip_threshold,
                support_low = support_low,
               bullish_ma = res_bullish_ma,
                bullish_candle = res_bullish_candle,
                candle_size = candle_size,
                touch_low = touch_low,
                midpoint_offset = res_midpoint_offset,
                bound_pips = bound_pips
               )

trade_strategy = 'support_indicator'
trade_strategy2 = 'resistance_indicator'

#trade_strategy = 'resistance_indicatorc'
num_trades = 400
risk_size = .01

starting_val = 10000
instrument = pair
timeframe = 'M5'
temp_lst = []
actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
actual_number_of_trades2 = df_[df_[trade_strategy2] == 1].shape[0]
for sl in sls:
    for tp in tps:
        ending_val,win_rate,wins,losses = strategy_tester_buy(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)
        ending_val2,win_rate2,wins2,losses2 = strategy_tester_sell(df_,col = trade_strategy2,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)

        lst = [instrument,
                timeframe,
                'buy',
                trade_strategy,
                ending_val,
                ending_val2,
               ending_val + ending_val2 - starting_val,
                starting_val,
                sl,
                tp,
                round(tp / sl,4),
                round(win_rate,4),
                num_trades,
                risk_size,
                actual_number_of_trades,
               'sell',
                trade_strategy2,

                win_rate2,
                actual_number_of_trades2,
                lookup_range,
                lookup_range2,
                stop_range,
                pip_threshold,
                support_low ,
                bullish_ma ,
                bullish_candle ,
                candle_size ,
                touch_low ,
                midpoint_offset,
                bound_pips 
        ]
        
        temp_lst.append(lst)
        
        
        ending_val,win_rate,wins,losses = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)
        ending_val2,win_rate2,wins2,losses2 = strategy_tester_buy(df_,col = trade_strategy2,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)

        lst = [instrument,
                timeframe,
                'sell',
                trade_strategy,
                ending_val,
                ending_val2,
               ending_val + ending_val2 - starting_val,
                starting_val,
                sl,
                tp,
                round(tp / sl,4),
                round(win_rate,4),
                num_trades,
                risk_size,
                actual_number_of_trades,
               'buy',
                trade_strategy2,
                win_rate2,
                actual_number_of_trades2,
                lookup_range,
                lookup_range2,
                stop_range,
                pip_threshold,
                support_low ,
                bullish_ma ,
                bullish_candle ,
                candle_size ,
                touch_low ,
                midpoint_offset,
                bound_pips 
        ]
        
        temp_lst.append(lst)
        
                
        
        
final_df = pd.DataFrame(temp_lst,columns = cols)
final_df = final_df.sort_values(by = 'median_sim_ending_val',ascending = False)
final_df.head(10)


In [ ]:
def run_single_parameter_sim_loop(df,
                                 df_,
                                  pair,
                                 lookup_range,
                                    lookup_range2,
                                    stop_range,
                                    pip_threshold,
                                    support_low,
                                   bullish_ma,
                                    bullish_candle,
                                    candle_size,
                                    touch_low,
                                    midpoint_offset,
                                    bound_pips,sls,tps):
    
    def strategy_tester_buy(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True):

        col_num = df.columns.get_loc(col)
        h_col = df.columns.get_loc('h')
        l_col = df.columns.get_loc('l')
        c_col = df.columns.get_loc('c')
        entry_col = df.columns.get_loc(entry)
        arr = df.values
        temp_df = df[df[col] == 1]
 
        trade_list = []
        trade_res = []
        wins = []
        losses = []
        for i in range(arr.shape[0]):

            if arr[i,col_num] == 1:
                val = arr[i,entry_col]
                trade_id = 0
                try:
                    for j in range(i + 1,i + 20000):
                        if i not in trade_list:
                            if trade_id == 0:
                                if arr[j,l_col] <= val - sl:
                                    trade_res.append(-10)
                                    trade_list.append(i)
                                    losses.append(i)
                                    trade_id = 1

                                    break
                                elif arr[j,h_col] >= val + tp:
                                    trade_res.append(50)
                                    trade_list.append(i)
                                    wins.append(i)
                                    trade_id = 1
                                    break
                except:
                    pass
        win_rate = len(wins) / len(trade_list)
        rr = tp / sl
        vals = []


        acc_val = 10000

        for i in range(30):

            final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
            vals.append(final_val)


        #print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
        return np.median(vals),win_rate,wins,losses

    def strategy_tester_sell(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True):

        col_num = df.columns.get_loc(col)
        h_col = df.columns.get_loc('h')
        l_col = df.columns.get_loc('l')
        c_col = df.columns.get_loc('c')
        entry_col = df.columns.get_loc(entry)
        arr = df.values
        temp_df = df[df[col] == 1]
   
        trade_list = []
        trade_res = []
        wins = []
        losses = []
        for i in range(arr.shape[0]):

            if arr[i,col_num] == 1:
                val = arr[i,entry_col]
                try:
                    for j in range(i + 1,i + 20000):
                        if i not in trade_list:
                            if arr[j,h_col] >= val + sl:
                                trade_res.append(-10)
                                trade_list.append(i)
                                losses.append(i)
                                break
                            elif arr[j,l_col] <= val - tp:
                                trade_res.append(50)
                                trade_list.append(i)
                                wins.append(i)
                                break
                except:
                    pass
        win_rate = len(wins) / len(trade_list)
        rr = tp / sl
        vals = []

        acc_val = 10000

        for i in range(30):

            final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
            vals.append(final_val)


        #print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
        return np.median(vals),win_rate,wins,losses
    
    
    def get_support(df,
                    lookup_range = 1500,
                    lookup_range2 = 2400,
                    stop_range = 2200,
                    pip_threshold = .005,
                    support_low = True,
                   bullish_ma = True,
                    bullish_candle = True,
                    candle_size = .001,
                    touch_low = True,
                    midpoint_offset = 0,
                    bound_pips = .001
                   ):
        print('GETTING SUPPORT INDICATOR')
        s = time.time()
        c_col = df.columns.get_loc('c')
        o_col = df.columns.get_loc('o')
        l_col = df.columns.get_loc('l')
        h_col = df.columns.get_loc('h')
        uw_col = df.columns.get_loc('upper_wick')
        lw_col = df.columns.get_loc('lower_wick')
        dv_col = df.columns.get_loc('delta_vector')
        dm_col = df.columns.get_loc('delta_max')
        d_col = df.columns.get_loc('delta')
        ma1_col = df.columns.get_loc('smma_21')
        ma2_col = df.columns.get_loc('smma_50')
        ma3_col = df.columns.get_loc('smma_200')    

        df['support_lookup'] = 0
        df['support_indicator'] = 0

        lookup_col = df.columns.get_loc('support_lookup')
        new_col = df.columns.get_loc('support_indicator')

        if support_low:
            reversal_col = df.columns.get_loc('l')
        else:
            reversal_col = df.columns.get_loc('c')

        if touch_low:
            touch_col = df.columns.get_loc('l')
        else:
            touch_col = df.columns.get_loc('c')    

        arr = df.values
        min_count = 0

        print('STOP RANGE:',stop_range)
        c = 0 

    #['time', 'volume', 'o', 'h', 'l', 'c', 'utc_timestamp', 'est_timestamp',
     #      'smma_21', 'smma_50', 'smma_200', 'ema_42', 'ema_150', 'ema_600',
      #     'bearish_engulfing', 'bullish_engulfing', 'bearish_tls', 'bullish_tls',
       #    'new_york', 'london', 'delta_vector', 'delta_max', 'upper_wick',
        #   'lower_wick', 'delta']

        for i in range(arr.shape[0]):
           # if i % 200000 == 0:
            #    print(i)
            try:
                #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
                min_ = min(arr[i - lookup_range : i + lookup_range2,reversal_col])

                # If the current val is at that reversal point:
                # Check and make sure we have an adequate movement off of that reversal point
                # this is defined by the pip_range and pip threshold, pip threshold is a user defined parameter
                if arr[i,reversal_col] == min_:
                    lookup_ind = i
                    val = arr[i,reversal_col]
                    lookup_check = 0
                    max_ = max(arr[i : i + lookup_range2,c_col])
                    pip_range = max_ - min_
                    c = c + 1
                    if pip_threshold <= pip_range:
                        # Now that all of these conditions are met
                        # We need to look into the future to see if we hit this reversal level, this is defined by the stop range
                        for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                            #If all of the following conditions are met, we will mark the array with a 1 signal to indicated we would place a trade                       
                            #CANDLE BULL OR BEAR
                            if arr[j,c_col] >= arr[j,o_col]:
                                bull = True
                            else:
                                bull = False
                            #MOVING AVG INDICATOR
                            if bullish_ma == True:
                                if arr[j,ma1_col] > arr[j,ma2_col] and arr[j,ma2_col] > arr[j,ma3_col]:
                                    ma_indicator = 1
                                else:
                                    ma_indicator = 0
                            elif bullish_ma == False:
                                if arr[j,ma1_col] < arr[j,ma2_col] and arr[j,ma2_col] < arr[j,ma3_col]:
                                    ma_indicator = 1
                                else:
                                    ma_indicator = 0                        
                            else:
                                ma_indicator = 1

                            midpoint = val + midpoint_offset
                            lower_bound = midpoint - bound_pips
                            upper_bound = midpoint + bound_pips

                            if lookup_check == 0 and \
                            ma_indicator == 1 and\
                            (bull == bullish_candle or bullish_candle == None) and\
                            arr[j,d_col] >= candle_size and\
                            (arr[j,touch_col] >= lower_bound and arr[j,touch_col] <= upper_bound):

                                arr[j,new_col] = 1
                                arr[j,lookup_col] = lookup_ind
                                lookup_check = 1
                                break

            except:
                pass

        print('COUNT:',c)

        e = time.time()
        print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
        df = pd.DataFrame(arr,columns = df.columns)
        print('SHAPE',df[df['support_indicator'] == 1].shape)
        return df


    def get_resistance(df,
                    lookup_range = 1500,
                    lookup_range2 = 2400,
                    stop_range = 2200,
                    pip_threshold = .005,
                    support_low = True,
                   bullish_ma = True,
                    bullish_candle = True,
                    candle_size = .001,
                    touch_low = True,
                    midpoint_offset = 0,
                    bound_pips = .001
                   ):
        print('GETTING RESISTANCE INDICATOR')
        s = time.time()
        c_col = df.columns.get_loc('c')
        o_col = df.columns.get_loc('o')
        l_col = df.columns.get_loc('l')
        h_col = df.columns.get_loc('h')
        uw_col = df.columns.get_loc('upper_wick')
        lw_col = df.columns.get_loc('lower_wick')
        dv_col = df.columns.get_loc('delta_vector')
        dm_col = df.columns.get_loc('delta_max')
        d_col = df.columns.get_loc('delta')
        ma1_col = df.columns.get_loc('smma_21')
        ma2_col = df.columns.get_loc('smma_50')
        ma3_col = df.columns.get_loc('smma_200')    

        df['resistance_lookup'] = 0
        df['resistance_indicator'] = 0

        lookup_col = df.columns.get_loc('resistance_lookup')
        new_col = df.columns.get_loc('resistance_indicator')

        if support_low:
            reversal_col = df.columns.get_loc('h')
        else:
            reversal_col = df.columns.get_loc('c')

        if touch_low:
            touch_col = df.columns.get_loc('h')
        else:
            touch_col = df.columns.get_loc('c')    

        arr = df.values
        min_count = 0
        c = 0
        for i in range(arr.shape[0]):

            try:
                #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
                max_ = max(arr[i - lookup_range : i + lookup_range2,reversal_col])

                # If the current val is at that reversal point:
                # Check and make sure we have an adequate movement off of that reversal point
                # this is defined by the pip_range and pip threshold, pip threshold is a user defined parameter
                if arr[i,reversal_col] == max_:
                    lookup_ind = i
                    val = arr[i,reversal_col]
                    lookup_check = 0
                    min_ = min(arr[i : i + lookup_range2,c_col])
                    pip_range = max_ - min_
                    c = c + 1
                    if pip_threshold <= pip_range:
                        # Now that all of these conditions are met
                        # We need to look into the future to see if we hit this reversal level, this is defined by the stop range
                        for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                            #If all of the following conditions are met, we will mark the array with a 1 signal to indicated we would place a trade                       
                            #CANDLE BULL OR BEAR
                            if arr[j,c_col] >= arr[j,o_col]:
                                bull = True
                            else:
                                bull = False
                            #MOVING AVG INDICATOR
                            if bullish_ma == True:
                                if arr[j,ma1_col] > arr[j,ma2_col] and arr[j,ma2_col] > arr[j,ma3_col]:
                                    ma_indicator = 1
                                else:
                                    ma_indicator = 0
                            elif bullish_ma == False:
                                if arr[j,ma1_col] < arr[j,ma2_col] and arr[j,ma2_col] < arr[j,ma3_col]:
                                    ma_indicator = 1
                                else:
                                    ma_indicator = 0                        
                            else:
                                ma_indicator = 1

                            midpoint = val + midpoint_offset
                            lower_bound = midpoint - bound_pips
                            upper_bound = midpoint + bound_pips

                            if lookup_check == 0 and \
                            ma_indicator == 1 and\
                            (bull == bullish_candle or bullish_candle == None) and\
                            arr[j,d_col] >= candle_size and\
                            (arr[j,touch_col] >= lower_bound and arr[j,touch_col] <= upper_bound):

                                arr[j,new_col] = 1
                                arr[j,lookup_col] = lookup_ind
                                lookup_check = 1
                                break

            except:
                pass

        e = time.time()
        df = pd.DataFrame(arr,columns = df.columns)
        print('SHAPE',df[df['resistance_indicator'] == 1].shape)
        return df

    s = time.time()
    print('RUNNING SINGLE PARAMETER LOOP FOR PARAMS:',
          'lookup_range',lookup_range,
          'lookup_range2', lookup_range2,
                    'stop_range', stop_range,
                    'pip_threshold' , pip_threshold,
                    'support_low' , support_low,
                   'bullish_ma' , bullish_ma,
                    'bullish_candle' , bullish_candle,
                    'candle_size' , candle_size,
                    'touch_low' , touch_low,
                    'midpoint_offset' , midpoint_offset,
                    'bound_pips' , bound_pips)
    
    cols = [
        'instrument'
        ,'timeframe'
        ,'buy_or_sell'
        ,'trade_strategy'
       ,'median_sim_ending_val'
        ,'median_sim_ending_val2'
        ,'total_sim_val'
        ,'starting_val'
        ,'sl'
        ,'tp'
        ,'RR'
        ,'win_rate'    
        ,'num_trades'
        ,'risk_size' 
        ,'actual_number_of_trades'   
        ,'buy_or_sell2'
        ,'trade_strategy2'

        ,'win_rate2'    
        ,'actual_number_of_trades2'    
        ,'lookup_range' 
        ,'lookup_range2' 
        ,'stop_range' 
        ,'pip_threshold' 
        ,'support_low' 
        ,'bullish_ma' 
        ,'bullish_candle'
        ,'candle_size' 
        ,'touch_low' 
        ,'midpoint_offset' 
        ,'bound_pips' 

    ]

    df = get_support(df,
                    lookup_range = lookup_range,
                    lookup_range2 = lookup_range2,
                    stop_range = stop_range,
                    pip_threshold = pip_threshold,
                    support_low = support_low,
                   bullish_ma = bullish_ma,
                    bullish_candle = bullish_candle,
                    candle_size = candle_size,
                    touch_low = touch_low,
                    midpoint_offset = midpoint_offset,
                    bound_pips = bound_pips
                   )

    if bullish_ma == True:
        res_bullish_ma = False
    elif bullish_ma == False:
        res_bullish_ma = True
    else:
        res_bullish_ma = None

    if bullish_candle == True:
        res_bullish_candle = False
    elif bullish_candle == False:
        res_bullish_candle = True
    else:
        res_bullish_candle = None

    res_midpoint_offset = midpoint_offset * -1

    df_ = get_resistance(df_,
                    lookup_range = lookup_range,
                    lookup_range2 = lookup_range2,
                    stop_range = stop_range,
                    pip_threshold = pip_threshold,
                    support_low = support_low,
                   bullish_ma = res_bullish_ma,
                    bullish_candle = res_bullish_candle,
                    candle_size = candle_size,
                    touch_low = touch_low,
                    midpoint_offset = res_midpoint_offset,
                    bound_pips = bound_pips
                   )

    trade_strategy = 'support_indicator'
    trade_strategy2 = 'resistance_indicator'

    #trade_strategy = 'resistance_indicatorc'
    num_trades = 400
    risk_size = .01

    starting_val = 10000
    instrument = pair
    timeframe = 'M5'
    temp_lst = []
    actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
    actual_number_of_trades2 = df_[df_[trade_strategy2] == 1].shape[0]
    for sl in sls:
        for tp in tps:
            ending_val,win_rate,wins,losses = strategy_tester_buy(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)
            ending_val2,win_rate2,wins2,losses2 = strategy_tester_sell(df_,col = trade_strategy2,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)

            lst = [instrument,
                    timeframe,
                    'buy',
                    trade_strategy,
                    ending_val,
                    ending_val2,
                   ending_val + ending_val2 - starting_val,
                    starting_val,
                    sl,
                    tp,
                    round(tp / sl,4),
                    round(win_rate,4),
                    num_trades,
                    risk_size,
                    actual_number_of_trades,
                   'sell',
                    trade_strategy2,

                    win_rate2,
                    actual_number_of_trades2,
                    lookup_range,
                    lookup_range2,
                    stop_range,
                    pip_threshold,
                    support_low ,
                    bullish_ma ,
                    bullish_candle ,
                    candle_size ,
                    touch_low ,
                    midpoint_offset,
                    bound_pips 
            ]

            temp_lst.append(lst)


            ending_val,win_rate,wins,losses = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)
            ending_val2,win_rate2,wins2,losses2 = strategy_tester_buy(df_,col = trade_strategy2,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)

            lst = [instrument,
                    timeframe,
                    'sell',
                    trade_strategy,
                    ending_val,
                    ending_val2,
                   ending_val + ending_val2 - starting_val,
                    starting_val,
                    sl,
                    tp,
                    round(tp / sl,4),
                    round(win_rate,4),
                    num_trades,
                    risk_size,
                    actual_number_of_trades,
                   'buy',
                    trade_strategy2,
                    win_rate2,
                    actual_number_of_trades2,
                    lookup_range,
                    lookup_range2,
                    stop_range,
                    pip_threshold,
                    support_low ,
                    bullish_ma ,
                    bullish_candle ,
                    candle_size ,
                    touch_low ,
                    midpoint_offset,
                    bound_pips 
            ]

            temp_lst.append(lst)




    final_df = pd.DataFrame(temp_lst,columns = cols)
    final_df = final_df.sort_values(by = 'total_sim_val',ascending = False)
    
    support_best = final_df.sort_values(by = 'median_sim_ending_val',ascending = False)
    resistance_best = final_df.sort_values(by = 'median_sim_ending_val2',ascending = False)
    overall_best = final_df.sort_values(by = 'total_sim_val',ascending = False)

    print('SUPPORT BEST BUY/VAL/WR/RR/TP/SL/NUM_TRADES:',support_best['buy_or_sell'].iloc[0],'/',support_best['median_sim_ending_val'].iloc[0],'/',support_best['win_rate'].iloc[0],'/',support_best['RR'].iloc[0],'/',support_best['tp'].iloc[0],'/',support_best['sl'].iloc[0],'/',support_best['actual_number_of_trades'].iloc[0]
         )
    print('RESISTANCE BEST BUY/VAL/WR/RR/TP/SL/NUM_TRADES:',resistance_best['buy_or_sell'].iloc[0],'/',resistance_best['median_sim_ending_val2'].iloc[0],'/',resistance_best['win_rate2'].iloc[0],'/',resistance_best['RR'].iloc[0],'/',resistance_best['tp'].iloc[0],'/',resistance_best['sl'].iloc[0],'/',resistance_best['actual_number_of_trades2'].iloc[0]
         )
    print('OVERALL BEST BUY/VAL/WR/RR/TP/SL/NUM_TRADES:',overall_best['buy_or_sell'].iloc[0],'/',overall_best['total_sim_val'].iloc[0],'/',overall_best['win_rate'].iloc[0],'/',overall_best['RR'].iloc[0],'/',overall_best['tp'].iloc[0],'/',overall_best['sl'].iloc[0],'/',overall_best['actual_number_of_trades'].iloc[0]
         )


    e = time.time()
    print('TOTAL PARAM SIM LOOP TIME:',round((e-s)/60,2),'MINUTES')
    return final_df


def generate_random_param_list():
    def return_random_list_element(lst):
        return lst[random.randint(0,len(lst) - 1)]
    # param loop
    # PARAM DEFINITON
    if 'JPY' in pair:
        pip_thresholds = [.3,.5,.7]
        sls = [.05,.075,.1,.125,.15,.2,.25]
        tps = [.1,.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3]
        candle_sizes = [.05,.075,.1,.15]
        midpoint_offsets = [0,-.05,.05,-.1,.1,-.15,.15,-.25,.25]
        bound_pips_ = [.025,.05,.075,.1,.15]
    else:
        pip_thresholds = [.003,.005,.007]
        sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
        tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013]  
        candle_sizes = [.0005,.00075,.001,.0015]
        midpoint_offsets = [0,-.0005,.0005,-.001,.001,-.0015,.0015,-.0025,.0025]
        bound_pips_ = [.00025,.0005,.00075,.001,.0015]

    lookup_ranges = [100,288,400,600]
    lookup_ranges2 = [100,288,400,600]    
    stop_ranges = [1000,3000,6000,8000]

    support_lows = [True,False]
    bullish_mas = [True,False,None]
    bullish_candles = [True,False,None]
    touch_lows = [True,False]
    
    lookup_range = return_random_list_element(lookup_ranges)
    lookup_range2 = return_random_list_element(lookup_ranges2)
    stop_range = return_random_list_element(stop_ranges)
    pip_threshold = return_random_list_element(pip_thresholds)
    support_low = return_random_list_element(support_lows)
    bullish_ma = return_random_list_element(bullish_mas)
    bullish_candle = return_random_list_element(bullish_candles)
    candle_size = return_random_list_element(candle_sizes)
    touch_low = return_random_list_element(touch_lows)
    midpoint_offset = return_random_list_element(midpoint_offsets)
    bound_pips = return_random_list_element(bound_pips_)
    param_check_list = [lookup_range,lookup_range2,stop_range,pip_threshold,support_low,bullish_ma,bullish_candle,candle_size,touch_low,midpoint_offset,bound_pips]
    
    return param_check_list,lookup_range,lookup_range2,stop_range,pip_threshold,support_low,bullish_ma,bullish_candle,candle_size,touch_low,midpoint_offset,bound_pips,sls,tps



def run_sim(file,max_loops):
    print('RUNNING SIM...')
    file_name = 'SIM_CLEAN_' + file
    file_name2 = 'CHECK_PARAMS_' + file
    try:
        final_df = pd.read_csv(file_name)
        loop_number = final_df['loop_number'].max()

        param_check_df = pd.read_csv(file_name2)
        param_check_lists = param_check_df.values.tolist()

    except:
        final_df = pd.DataFrame([0])
        param_check_df = pd.DataFrame([0])
        loop_number = 0
        param_check_lists = []
    
    while max_loops >= loop_number:
        checker = 0
        while checker == 0:
            param_check_list,lookup_range,lookup_range2,stop_range,pip_threshold,support_low,bullish_ma,bullish_candle,candle_size,touch_low,midpoint_offset,bound_pips,sls,tps = generate_random_param_list()
            if param_check_list not in param_check_lists:
                param_check_lists.append(param_check_list)

                checker = 1
        try:
            if final_df.shape[0] == 1:
                loop_number = 1
            else:
                loop_number = final_df['loop_number'].max() + 1

            print('LOOP NUMBER:',loop_number)
            loop_df = run_single_parameter_sim_loop(df,
                                                 df_,
                                                  pair,
                                                 lookup_range,
                                                    lookup_range2,
                                                    stop_range,
                                                    pip_threshold,
                                                    support_low,
                                                   bullish_ma,
                                                    bullish_candle,
                                                    candle_size,
                                                    touch_low,
                                                    midpoint_offset,
                                                    bound_pips,sls,tps)
            loop_df['loop_number'] = loop_number

            if final_df.shape[0] == 1:
                final_df = loop_df

            else:
                final_df = final_df.append(loop_df)

            final_df.to_csv(file_name,index = False)
            param_check_df = pd.DataFrame(param_check_lists)
            param_check_df.to_csv(file_name2,index = False)
        except:
            print('ERROR WITH THESE PARAMETERS... SKIPPING')

        support_best = final_df.sort_values(by = 'median_sim_ending_val',ascending = False)
        resistance_best = final_df.sort_values(by = 'median_sim_ending_val2',ascending = False)
        overall_best = final_df.sort_values(by = 'total_sim_val',ascending = False)

        print('OVR SUPPORT BEST BUY/VAL/WR/RR/TP/SL/NUM_TRADES:',support_best['buy_or_sell'].iloc[0],'/',support_best['median_sim_ending_val'].iloc[0],'/',support_best['win_rate'].iloc[0],'/',support_best['RR'].iloc[0],'/',support_best['tp'].iloc[0],'/',support_best['sl'].iloc[0],'/',support_best['actual_number_of_trades'].iloc[0]
             )
        print('OVR RESISTANCE BEST BUY/VAL/WR/RR/TP/SL/NUM_TRADES:',resistance_best['buy_or_sell'].iloc[0],'/',resistance_best['median_sim_ending_val2'].iloc[0],'/',resistance_best['win_rate2'].iloc[0],'/',resistance_best['RR'].iloc[0],'/',resistance_best['tp'].iloc[0],'/',resistance_best['sl'].iloc[0],'/',resistance_best['actual_number_of_trades2'].iloc[0]
             )
        print('OVR OVERALL BEST BUY/VAL/WR/RR/TP/SL/NUM_TRADES:',overall_best['buy_or_sell'].iloc[0],'/',overall_best['total_sim_val'].iloc[0],'/',overall_best['win_rate'].iloc[0],'/',overall_best['RR'].iloc[0],'/',overall_best['tp'].iloc[0],'/',overall_best['sl'].iloc[0],'/',overall_best['actual_number_of_trades'].iloc[0]
             )
        print()
        print()
        print()
        

run_sim(file,max_loops = 5000)




SUPPORT BEST BUY/VAL/WR/RR/TP/SL/NUM_TRADES: buy / 61644.0 / 0.3034 / 4.0 / 0.003 / 0.00075 / 89
RESISTANCE BEST BUY/VAL/WR/RR/TP/SL/NUM_TRADES: buy / 23654.0 / 0.40625 / 2.0 / 0.005 / 0.0025 / 96
OVERALL BEST BUY/VAL/WR/RR/TP/SL/NUM_TRADES: buy / 65026.5 / 0.3034 / 4.0 / 0.003 / 0.00075 / 89
TOTAL PARAM SIM LOOP TIME: 0.72 MINUTES
OVR SUPPORT BEST BUY/VAL/WR/RR/TP/SL/NUM_TRADES: sell / 1.4067536382281777e+44 / 1.0 / 26.0 / 0.013 / 0.0005 / 1
OVR RESISTANCE BEST BUY/VAL/WR/RR/TP/SL/NUM_TRADES: buy / 1.00968472873817e+23 / 0.5 / 26.0 / 0.013 / 0.0005 / 2
OVR OVERALL BEST BUY/VAL/WR/RR/TP/SL/NUM_TRADES: sell / 1.4067536382281777e+44 / 1.0 / 26.0 / 0.013 / 0.0005 / 1



LOOP NUMBER: 805
RUNNING SINGLE PARAMETER LOOP FOR PARAMS: lookup_range 100 lookup_range2 400 stop_range 6000 pip_threshold 0.005 support_low False bullish_ma False bullish_candle False candle_size 0.0005 touch_low True midpoint_offset 0.001 bound_pips 0.001
GETTING SUPPORT INDICATOR
STOP RANGE: 6000
COUNT: 820
TOTAL FUNCT

In [ ]:
final_df.tail()

In [ ]:
# param loop
# PARAM DEFINITON
if 'JPY' in pair:
    pip_thresholds = [.3,.5,.7]
    sls = [.05,.075,.1,.125,.15,.2,.25]
    tps = [.1,.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3]
    candle_sizes = [.05,.075,.1,.15]
    midpoint_offsets = [0,-.05,.05,-.1,.1,-.15,.15,-.25,.25]
    bound_pips_ = [.025,.05,.075,.1,.15]
else:
    pip_thresholds = [.003,.005,.007]
    sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
    tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013]  
    candle_sizes = [.0005,.00075,.001,.0015]
    midpoint_offsets = [0,-.0005,.0005,-.001,.001,-.0015,.0015,-.0025,.0025]
    bound_pips_ = [.00025,.0005,.00075,.001,.0015]
    
lookup_ranges = [100,288,400,600]
lookup_ranges2 = [100,288,400,600]    
stop_ranges = [1000,3000,6000,8000]

support_lows = [True,False]
bullish_mas = [True,False,None]
bullish_candles = [True,False,None]
touch_lows = [True,False]

lookup_range = 250
lookup_range2 = 150
stop_range = 4500
pip_threshold = pip_thresholds[1]
support_low = True
bullish_ma = None
bullish_candle = False
candle_size = candle_sizes[2]
touch_low = touch_lows[0]
midpoint_offset = midpoint_offsets[0]
bound_pips = bound_pips_[3]


def return_random_list_element(lst):
    return lst[random.randint(0,len(lst) - 1)]
def generate_random_param_list():
    lookup_range = return_random_list_element(lookup_ranges)
    lookup_range2 = return_random_list_element(lookup_ranges2)
    stop_range = return_random_list_element(stop_ranges)
    pip_threshold = return_random_list_element(pip_thresholds)
    support_low = return_random_list_element(support_lows)
    bullish_ma = return_random_list_element(bullish_mas)
    bullish_candle = return_random_list_element(bullish_candles)
    candle_size = return_random_list_element(candle_sizes)
    touch_low = return_random_list_element(touch_lows)
    midpoint_offset = return_random_list_element(midpoint_offsets)
    bound_pips = return_random_list_element(bound_pips_)
    param_check_list = [lookup_range,lookup_range2,stop_range,pip_threshold,support_low,bullish_ma,bullish_candle,candle_size,touch_low,midpoint_offset,bound_pips]
    return param_check_list
        
        
param_check_lists = []
keep_looping = True

while keep_looping:
    lookup_range = return_random_list_element(lookup_ranges)
    lookup_range2 = return_random_list_element(lookup_ranges2)
    stop_range = return_random_list_element(stop_ranges)
    pip_threshold = return_random_list_element(pip_thresholds)
    support_low = return_random_list_element(support_lows)
    bullish_ma = return_random_list_element(bullish_mas)
    bullish_candle = return_random_list_element(bullish_candles)
    candle_size = return_random_list_element(candle_sizes)
    touch_low = return_random_list_element(touch_lows)
    midpoint_offset = return_random_list_element(midpoint_offsets)
    bound_pips = return_random_list_element(bound_pips_)
    
    param_check_list = [lookup_range,lookup_range2,stop_range,pip_threshold,support_low,bullish_ma,bullish_candle,candle_size,touch_low,midpoint_offset,bound_pips]
    if param_check_list not in param_check_lists:
        
        
        
        param_check_lists.append(param_check_list)
    if len(param_check_lists) > 10:
        keep_looping = False
param_check_lists

In [ ]:

random.randint(0,10)

In [ ]:
def generate_random_param_list():
    def return_random_list_element(lst):
        return lst[random.randint(0,len(lst) - 1)]
    # param loop
    # PARAM DEFINITON
    if 'JPY' in pair:
        pip_thresholds = [.3,.5,.7]
        sls = [.05,.075,.1,.125,.15,.2,.25]
        tps = [.1,.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3]
        candle_sizes = [.05,.075,.1,.15]
        midpoint_offsets = [0,-.05,.05,-.1,.1,-.15,.15,-.25,.25]
        bound_pips_ = [.025,.05,.075,.1,.15]
    else:
        pip_thresholds = [.003,.005,.007]
        sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
        tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013]  
        candle_sizes = [.0005,.00075,.001,.0015]
        midpoint_offsets = [0,-.0005,.0005,-.001,.001,-.0015,.0015,-.0025,.0025]
        bound_pips_ = [.00025,.0005,.00075,.001,.0015]

    lookup_ranges = [100,288,400,600]
    lookup_ranges2 = [100,288,400,600]    
    stop_ranges = [1000,3000,6000,8000]

    support_lows = [True,False]
    bullish_mas = [True,False,None]
    bullish_candles = [True,False,None]
    touch_lows = [True,False]
    
    lookup_range = return_random_list_element(lookup_ranges)
    lookup_range2 = return_random_list_element(lookup_ranges2)
    stop_range = return_random_list_element(stop_ranges)
    pip_threshold = return_random_list_element(pip_thresholds)
    support_low = return_random_list_element(support_lows)
    bullish_ma = return_random_list_element(bullish_mas)
    bullish_candle = return_random_list_element(bullish_candles)
    candle_size = return_random_list_element(candle_sizes)
    touch_low = return_random_list_element(touch_lows)
    midpoint_offset = return_random_list_element(midpoint_offsets)
    bound_pips = return_random_list_element(bound_pips_)
    param_check_list = [lookup_range,lookup_range2,stop_range,pip_threshold,support_low,bullish_ma,bullish_candle,candle_size,touch_low,midpoint_offset,bound_pips]
    
    return param_check_list,lookup_range,lookup_range2,stop_range,pip_threshold,support_low,bullish_ma,bullish_candle,candle_size,touch_low,midpoint_offset,bound_pips

param_check_lists = []

checker = 0
while checker == 0:
    param_check_list,lookup_range,lookup_range2,stop_range,pip_threshold,support_low,bullish_ma,bullish_candle,candle_size,touch_low,midpoint_offset,bound_pips = generate_random_param_list()
    if param_check_list not in param_check_lists:
        param_check_lists.append(param_check_list)
        
        checker = 1
        
param_check_lists



In [ ]:
x = None
if x == True:
    print('1')
if x == False:
    print('2')
else:
    print('3')


In [ ]:

if 'JPY' in pair:
    pip_threshold = .3
    pip_threshold = .1
    pip_threshold2 = 0
    sls = [.05,.075,.1,.125,.15,.2,.25]
    tps = [.1,.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3]
else:
    pip_threshold = .005
    pip_threshold2 = .0025
    sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
    tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013]  
#pip_threshold2s = [.0005,.001,.0015,.002,.0025]
#pip_thresholds = [.003,.005,.0075]
#lookup_ranges = [500,1000,1500,2000,2500]
#lookup_range2s = [500,1000,1500,2000,2500]
#stop_ranges = [12500,10000,7500,5000,2500]


cols = [
    'instrument'
    ,'timeframe'
    ,'buy_or_sell'
    ,'trade_strategy'
   ,'median_sim_ending_val'
    ,'starting_val'
    ,'sl'
    ,'tp'
    ,'RR'
    ,'win_rate'
    ,'pip_threshold'
    ,'pip_threshold2'
    ,'lookup_range'
    ,'lookup_range2'
    ,'stop_range'
    ,'num_trades'
    ,'risk_size' 
    ,'actual_number_of_trades'
    
]


lookup_range = 288 # 1 day
lookup_range = 100
lookup_range2 = 288 # 1 day
lookup_range2 = 100
stop_range = 4000

signal = 'bullish_engulfing'

signal = 'bullish_tls'
signal = 'bearish_tls'
signal = 'bearish_engulfing'
signal = 'bearish_tls'
signal = 'bullish_tls'
signal = 'bearish_engulfing'
signal = 'bullish_engulfing'



buy_or_sell = 1
df = get_support(df,lookup_range = lookup_range,lookup_range2 = lookup_range2,stop_range = stop_range,
                  pip_threshold = pip_threshold,pip_threshold2 = pip_threshold2,trading_session = 'london',signal =  signal)
    

trade_strategy = 'support_indicator'
#trade_strategy = 'resistance_indicatorc'
num_trades = 400
risk_size = .01
actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
starting_val = 10000
instrument = pair
timeframe = 'M5'
temp_lst = []
for sl in sls:
    for tp in tps:
        ending_val,win_rate,wins,losses = strategy_tester_buy(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)

        lst = [instrument,timeframe,'buy',trade_strategy,ending_val,starting_val,sl,tp,round(tp / sl,4),round(win_rate,4),
         pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,num_trades,risk_size,actual_number_of_trades]
        temp_lst.append(lst)

        ending_val,win_rate,wins,losses = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False)

        lst = [instrument,timeframe,'sell',trade_strategy,ending_val,starting_val,sl,tp,round(tp / sl,4),round(win_rate,4),
         pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,num_trades,risk_size,actual_number_of_trades]
        temp_lst.append(lst)

final_df = pd.DataFrame(temp_lst,columns = cols)
final_df = final_df.sort_values(by = 'median_sim_ending_val',ascending = False)
final_df.head(10)

In [ ]:
print(df.columns)